In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
from pandas import DateOffset
import boto3
import sagemaker
from botocore.exceptions import ClientError
import logging
import matplotlib.pyplot as plt
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\pydantic\_internal\_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/23/25 13:44:29] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=152078;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=310607;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\botocore\credentials.py#1352\1352]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Usuario\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
from tqdm import tqdm

In [3]:
df=pd.read_csv('data_csv/27_materiales_mod.csv',sep=',',index_col=0,parse_dates=True,decimal='.')

In [30]:
materiales = df['material'].unique()
print(f'Los materiales son {materiales}')

Los materiales son [20000337001 20000400003 20000815002 20000815003 20000837001 20000837002
 20001016001 20001374001 20001497002 20001770001 20003147001 20003257001
 20003257002 20003257004 20003259001 20006083001 20007769001 20008046001
 20008540001 25101938001 25101940001 25109220001 25109223001 25109225001
 25109232001 25109241001 25110068001]


In [31]:
timeseries = []
materiales = df['material'].unique()
for mat in materiales:
    serie = df[df['material'] == mat].sort_index()
    timeseries.append(serie)

In [32]:
val_negative = df[df['cantidad'] < 0]
df_val_negative = val_negative[['cantidad', 'material']]
#print(f'Valores negativos:\n{df_val_negative}')
print(f'Cantidad de valores negativos: {len(df_val_negative)}')

Cantidad de valores negativos: 0


In [33]:
def check_frequencies(timeseries, infer_when_missing=True):
    """
    Revisa la frecuencia de cada DataFrame en la lista `timeseries`.

    Parameters
    ----------
    timeseries : list[pd.DataFrame] | list[pd.Series]
        Lista de objetos con índice de fechas.
    infer_when_missing : bool, default True
        Si la frecuencia no está definida, intenta inferirla con `pd.infer_freq`.

    Returns
    -------
    dict
        Mapeo {posición_en_lista: frecuencia_detectada (str | None)}
    """
    freqs = {}
    for i, ts in enumerate(timeseries):
        if not isinstance(ts.index, pd.DatetimeIndex):
            raise TypeError(f"Elemento {i} no tiene DatetimeIndex")

        # frecuencia explícita (pandas la guarda en .freq)
        freq = ts.index.freq

        # si no hay frecuencia y se pide inferir
        if freq is None and infer_when_missing:
            freq = pd.infer_freq(ts.index)

        freqs[i] = str(freq) if freq is not None else None
        print(f"timeseries[{i}] → {freq}")

    return freqs

In [34]:
freqs = check_frequencies(timeseries)

timeseries[0] → None
timeseries[1] → None
timeseries[2] → None
timeseries[3] → None
timeseries[4] → None
timeseries[5] → None
timeseries[6] → None
timeseries[7] → None
timeseries[8] → None
timeseries[9] → None
timeseries[10] → None
timeseries[11] → None
timeseries[12] → None
timeseries[13] → None
timeseries[14] → None
timeseries[15] → None
timeseries[16] → None
timeseries[17] → None
timeseries[18] → None
timeseries[19] → None
timeseries[20] → None
timeseries[21] → None
timeseries[22] → None
timeseries[23] → None
timeseries[24] → None
timeseries[25] → None
timeseries[26] → None


In [35]:
def completar_series_temporales_list(timeseries_list, freq='D'):
    """
    Completa las series temporales agregando fechas faltantes con cantidad=NaN.
    Mantiene los valores de las otras columnas para las fechas agregadas.
    
    Args:
        timeseries_list: Lista de DataFrames con índice 'fecha' y columna 'cantidad'
        freq: Frecuencia para completar ('D' para diario, 'W' para semanal, etc.)
    
    Returns:
        Lista con DataFrames completados
    """
    resultado = []
    
    for i, df in enumerate(tqdm(timeseries_list, desc="Completando series temporales")):
        # Verificar que es un DataFrame válido
        if not isinstance(df, pd.DataFrame) or df.empty:
            resultado.append(df)
            continue
        
        df_copy = df.copy()
        
        # Verificar que el índice es 'fecha' o convertirlo
        if df_copy.index.name != 'fecha':
            if 'fecha' in df_copy.columns:
                df_copy.set_index('fecha', inplace=True)
            else:
                print(f"DataFrame {i}: No se encontró la columna o índice 'fecha'")
                resultado.append(df)
                continue
        
        # Verificar que existe la columna 'cantidad'
        if 'cantidad' not in df_copy.columns:
            print(f"DataFrame {i}: No se encontró la columna 'cantidad'")
            resultado.append(df)
            continue
        
        # Asegurarse de que el índice es datetime
        if not isinstance(df_copy.index, pd.DatetimeIndex):
            df_copy.index = pd.to_datetime(df_copy.index)
        
        # Si el DataFrame está vacío después de las verificaciones, continuar
        if df_copy.empty:
            resultado.append(df_copy)
            continue
        
        # Obtener el rango completo de fechas
        fecha_min = df_copy.index.min()
        fecha_max = df_copy.index.max()
        
        # Crear el índice completo de fechas
        fechas_completas = pd.date_range(start=fecha_min, end=fecha_max, freq=freq)
        
        # Reindexar el DataFrame para incluir todas las fechas
        df_completo = df_copy.reindex(fechas_completas)
        
        # Asignar nombre al índice
        df_completo.index.name = 'fecha'
        
        # Para las fechas nuevas (que no existían), completar las columnas que no son 'cantidad'
        # con los valores más comunes o un valor de referencia
        for columna in df_completo.columns:
            if columna != 'cantidad':
                # Buscar el valor más común (moda) para esta columna
                valores_no_nulos = df_copy[columna].dropna()
                
                if not valores_no_nulos.empty:
                    if valores_no_nulos.dtype in ['object', 'string']:
                        # Para columnas categóricas/texto, usar la moda
                        valor_relleno = valores_no_nulos.mode().iloc[0] if len(valores_no_nulos.mode()) > 0 else valores_no_nulos.iloc[0]
                    else:
                        # Para columnas numéricas, usar la moda o la mediana si no hay moda clara
                        moda = valores_no_nulos.mode()
                        if len(moda) > 0:
                            valor_relleno = moda.iloc[0]
                        else:
                            valor_relleno = valores_no_nulos.median()
                    
                    # Rellenar solo los valores NaN (fechas nuevas)
                    df_completo[columna] = df_completo[columna].fillna(valor_relleno)
        
        # La columna 'cantidad' ya debe tener NaN para las fechas nuevas (esto es lo que queremos)
        
        print(f"DataFrame {i}: {len(df_copy)} → {len(df_completo)} registros ({len(df_completo) - len(df_copy)} fechas agregadas)")
        
        resultado.append(df_completo)
    
    return resultado

In [36]:
timeseries = completar_series_temporales_list(timeseries)

Completando series temporales:  85%|████████▌ | 23/27 [00:00<00:00, 229.13it/s]

DataFrame 0: 525 → 1370 registros (845 fechas agregadas)
DataFrame 1: 326 → 1353 registros (1027 fechas agregadas)
DataFrame 2: 551 → 1374 registros (823 fechas agregadas)
DataFrame 3: 544 → 1370 registros (826 fechas agregadas)
DataFrame 4: 399 → 1372 registros (973 fechas agregadas)
DataFrame 5: 524 → 1360 registros (836 fechas agregadas)
DataFrame 6: 528 → 1373 registros (845 fechas agregadas)
DataFrame 7: 217 → 1339 registros (1122 fechas agregadas)
DataFrame 8: 633 → 1323 registros (690 fechas agregadas)
DataFrame 9: 637 → 1366 registros (729 fechas agregadas)
DataFrame 10: 916 → 1373 registros (457 fechas agregadas)
DataFrame 11: 623 → 1366 registros (743 fechas agregadas)
DataFrame 12: 753 → 1366 registros (613 fechas agregadas)
DataFrame 13: 476 → 1374 registros (898 fechas agregadas)
DataFrame 14: 1023 → 1374 registros (351 fechas agregadas)
DataFrame 15: 741 → 1368 registros (627 fechas agregadas)
DataFrame 16: 611 → 1349 registros (738 fechas agregadas)
DataFrame 17: 466 → 1

Completando series temporales: 100%|██████████| 27/27 [00:00<00:00, 228.00it/s]

DataFrame 23: 689 → 1359 registros (670 fechas agregadas)
DataFrame 24: 546 → 1358 registros (812 fechas agregadas)
DataFrame 25: 633 → 1347 registros (714 fechas agregadas)
DataFrame 26: 681 → 1232 registros (551 fechas agregadas)


In [37]:
freqs = check_frequencies(timeseries)

timeseries[0] → <Day>
timeseries[1] → <Day>
timeseries[2] → <Day>
timeseries[3] → <Day>
timeseries[4] → <Day>
timeseries[5] → <Day>
timeseries[6] → <Day>
timeseries[7] → <Day>
timeseries[8] → <Day>
timeseries[9] → <Day>
timeseries[10] → <Day>
timeseries[11] → <Day>
timeseries[12] → <Day>
timeseries[13] → <Day>
timeseries[14] → <Day>
timeseries[15] → <Day>
timeseries[16] → <Day>
timeseries[17] → <Day>
timeseries[18] → <Day>
timeseries[19] → <Day>
timeseries[20] → <Day>
timeseries[21] → <Day>
timeseries[22] → <Day>
timeseries[23] → <Day>
timeseries[24] → <Day>
timeseries[25] → <Day>
timeseries[26] → <Day>


In [38]:
def sumar_cantidades_fechas(lista_dataframes, fechas=['2023-11-01', '2023-11-02']):
    """
    Suma la columna 'cantidad' para las fechas especificadas en cada dataframe
    
    Args:
        lista_dataframes: Lista de dataframes con índice de fecha
        fechas: Lista de fechas a considerar (por defecto '2023-11-01' y '2023-11-02')
    
    Returns:
        Un diccionario con las sumas por cada dataframe
    """
    resultados = {}
    
    for i, df in enumerate(lista_dataframes):
        suma = 0
        for fecha in fechas:
            if fecha in df.index:
                suma += df.loc[fecha, 'cantidad']
        resultados[f'dataframe_{i}'] = suma
    
    return resultados

In [39]:
resultados = sumar_cantidades_fechas(timeseries)

In [40]:
from tqdm import tqdm

In [41]:
def clean_outliers_timeseries_list(timeseries_list, z_score_threshold=3, excluded_dates=None):
    """
    Identifica y marca como NaN los valores atípicos en la columna 'cantidad'
    basándose en el z-score calculado por día de la semana (lunes a domingo).
    Excluye fechas específicas del análisis.
    
    Args:
        timeseries_list: Lista de DataFrames con índice 'fecha' y columna 'cantidad'
        z_score_threshold: Umbral de z-score para considerar un valor como outlier
        excluded_dates: Lista de fechas a excluir del análisis de outliers
        
    Returns:
        Lista con los DataFrames procesados
    """
    # Convertir fechas excluidas a datetime si se proporcionan
    if excluded_dates is None:
        excluded_dates = ['2023-11-01', '2023-11-02']  # Fechas por defecto a excluir
    
    excluded_dates = [pd.to_datetime(date) for date in excluded_dates]
    
    # Lista para almacenar resultados
    cleaned_series = []
    
    # Procesar cada DataFrame
    for i, df in enumerate(tqdm(timeseries_list, desc="Limpiando outliers por día de semana")):
        # Verificar que es un DataFrame válido
        if not isinstance(df, pd.DataFrame) or df.empty:
            cleaned_series.append(df)
            continue
        
        # Hacer una copia para no modificar el original
        df_copy = df.copy()
        
        # Resetear índice si 'fecha' está como índice
        if df_copy.index.name == 'fecha':
            df_copy = df_copy.reset_index()
        elif 'fecha' not in df_copy.columns and hasattr(df_copy.index, 'name'):
            # Si el índice parece ser fecha pero no tiene nombre
            df_copy = df_copy.reset_index()
            if 'index' in df_copy.columns:
                df_copy.rename(columns={'index': 'fecha'}, inplace=True)
        
        # Verificar que existe la columna 'cantidad'
        if 'cantidad' not in df_copy.columns:
            print(f"DataFrame {i}: No se encontró la columna 'cantidad'")
            cleaned_series.append(df)
            continue
        
        # Asegurarse de que la fecha es datetime
        if df_copy['fecha'].dtype != 'datetime64[ns]':
            df_copy['fecha'] = pd.to_datetime(df_copy['fecha'])
        
        # Crear columna de día de la semana (0=lunes, 6=domingo)
        df_copy['weekday'] = df_copy['fecha'].dt.dayofweek
        
        # Crear máscara para excluir fechas específicas
        exclude_mask = df_copy['fecha'].isin(excluded_dates)
        
        # Datos para análisis (excluyendo fechas específicas y valores NaN existentes)
        analysis_df = df_copy[~exclude_mask].copy()
        
        # Calcular z-scores para cada día de la semana
        outlier_indices = []
        
        for weekday in range(7):  # 0=lunes, 6=domingo
            # Filtrar datos para este día de la semana
            day_mask = analysis_df['weekday'] == weekday
            day_data = analysis_df.loc[day_mask, 'cantidad']
            
            # Si no hay suficientes datos (al menos 5) o todos son NaN, continuar
            if len(day_data) < 5 or day_data.isna().all():
                continue
            
            # Calcular media y desviación estándar (ignorando NaN)
            day_mean = day_data.mean()
            day_std = day_data.std()
            
            # Evitar división por cero
            if day_std == 0 or pd.isna(day_std):
                continue
            
            # Calcular z-scores: (valor - media) / desviación estándar
            z_scores = (day_data - day_mean) / day_std
            
            # Identificar outliers (z-score absoluto mayor que umbral)
            day_outliers = day_mask & z_scores.abs().gt(z_score_threshold)
            
            # Guardar índices de outliers
            if day_outliers.any():
                outlier_indices.extend(analysis_df[day_outliers].index.tolist())
        
        # Reemplazar outliers con NaN
        if outlier_indices:
            df_copy.loc[outlier_indices, 'cantidad'] = np.nan
            print(f"DataFrame {i}: Se identificaron {len(outlier_indices)} outliers de {len(df_copy)} registros")
        
        # Eliminar columna temporal weekday
        df_copy.drop('weekday', axis=1, inplace=True)
        
        # Restaurar el índice original si era necesario
        if df.index.name == 'fecha':
            df_copy.set_index('fecha', inplace=True)
        
        # Guardar DataFrame procesado
        cleaned_series.append(df_copy)
    
    return cleaned_series

In [42]:
timeseries = clean_outliers_timeseries_list(timeseries, z_score_threshold=3)

Limpiando outliers por día de semana:   0%|          | 0/27 [00:00<?, ?it/s]

DataFrame 0: Se identificaron 9 outliers de 1370 registros
DataFrame 1: Se identificaron 7 outliers de 1353 registros
DataFrame 2: Se identificaron 12 outliers de 1374 registros
DataFrame 3: Se identificaron 12 outliers de 1370 registros
DataFrame 4: Se identificaron 9 outliers de 1372 registros
DataFrame 5: Se identificaron 12 outliers de 1360 registros


Limpiando outliers por día de semana:  33%|███▎      | 9/27 [00:00<00:00, 79.99it/s]

DataFrame 6: Se identificaron 13 outliers de 1373 registros
DataFrame 7: Se identificaron 4 outliers de 1339 registros
DataFrame 8: Se identificaron 8 outliers de 1323 registros


Limpiando outliers por día de semana:  67%|██████▋   | 18/27 [00:00<00:00, 84.23it/s]

DataFrame 9: Se identificaron 13 outliers de 1366 registros
DataFrame 10: Se identificaron 21 outliers de 1373 registros
DataFrame 11: Se identificaron 13 outliers de 1366 registros
DataFrame 12: Se identificaron 20 outliers de 1366 registros
DataFrame 13: Se identificaron 15 outliers de 1374 registros
DataFrame 14: Se identificaron 20 outliers de 1374 registros
DataFrame 15: Se identificaron 17 outliers de 1368 registros
DataFrame 16: Se identificaron 13 outliers de 1349 registros
DataFrame 17: Se identificaron 12 outliers de 1316 registros
DataFrame 18: Se identificaron 12 outliers de 1092 registros
DataFrame 19: Se identificaron 16 outliers de 1166 registros
DataFrame 20: Se identificaron 14 outliers de 904 registros
DataFrame 21: Se identificaron 14 outliers de 1049 registros
DataFrame 22: Se identificaron 12 outliers de 1345 registros
DataFrame 23: Se identificaron 14 outliers de 1359 registros
DataFrame 24: Se identificaron 17 outliers de 1358 registros
DataFrame 25: Se identific

Limpiando outliers por día de semana: 100%|██████████| 27/27 [00:00<00:00, 81.37it/s]

DataFrame 26: Se identificaron 15 outliers de 1232 registros


In [43]:
timeseries = completar_series_temporales_list(timeseries)

Completando series temporales:   0%|          | 0/27 [00:00<?, ?it/s]

DataFrame 0: 1370 → 1370 registros (0 fechas agregadas)
DataFrame 1: 1353 → 1353 registros (0 fechas agregadas)
DataFrame 2: 1374 → 1374 registros (0 fechas agregadas)
DataFrame 3: 1370 → 1370 registros (0 fechas agregadas)
DataFrame 4: 1372 → 1372 registros (0 fechas agregadas)
DataFrame 5: 1360 → 1360 registros (0 fechas agregadas)
DataFrame 6: 1373 → 1373 registros (0 fechas agregadas)
DataFrame 7: 1339 → 1339 registros (0 fechas agregadas)
DataFrame 8: 1323 → 1323 registros (0 fechas agregadas)
DataFrame 9: 1366 → 1366 registros (0 fechas agregadas)
DataFrame 10: 1373 → 1373 registros (0 fechas agregadas)
DataFrame 11: 1366 → 1366 registros (0 fechas agregadas)
DataFrame 12: 1366 → 1366 registros (0 fechas agregadas)


Completando series temporales: 100%|██████████| 27/27 [00:00<00:00, 233.79it/s]

DataFrame 13: 1374 → 1374 registros (0 fechas agregadas)
DataFrame 14: 1374 → 1374 registros (0 fechas agregadas)
DataFrame 15: 1368 → 1368 registros (0 fechas agregadas)
DataFrame 16: 1349 → 1349 registros (0 fechas agregadas)
DataFrame 17: 1316 → 1316 registros (0 fechas agregadas)
DataFrame 18: 1092 → 1092 registros (0 fechas agregadas)
DataFrame 19: 1166 → 1166 registros (0 fechas agregadas)
DataFrame 20: 904 → 904 registros (0 fechas agregadas)
DataFrame 21: 1049 → 1049 registros (0 fechas agregadas)
DataFrame 22: 1345 → 1345 registros (0 fechas agregadas)
DataFrame 23: 1359 → 1359 registros (0 fechas agregadas)
DataFrame 24: 1358 → 1358 registros (0 fechas agregadas)
DataFrame 25: 1347 → 1347 registros (0 fechas agregadas)
DataFrame 26: 1232 → 1232 registros (0 fechas agregadas)


In [44]:
freqs = check_frequencies(timeseries)

timeseries[0] → <Day>
timeseries[1] → <Day>
timeseries[2] → <Day>
timeseries[3] → <Day>
timeseries[4] → <Day>
timeseries[5] → <Day>
timeseries[6] → <Day>
timeseries[7] → <Day>
timeseries[8] → <Day>
timeseries[9] → <Day>
timeseries[10] → <Day>
timeseries[11] → <Day>
timeseries[12] → <Day>
timeseries[13] → <Day>
timeseries[14] → <Day>
timeseries[15] → <Day>
timeseries[16] → <Day>
timeseries[17] → <Day>
timeseries[18] → <Day>
timeseries[19] → <Day>
timeseries[20] → <Day>
timeseries[21] → <Day>
timeseries[22] → <Day>
timeseries[23] → <Day>
timeseries[24] → <Day>
timeseries[25] → <Day>
timeseries[26] → <Day>


In [45]:
def handle_anomalous_dates_list(timeseries_list, anomalous_dates=['2023-11-01', '2023-11-02']):
    """
    Guarda y reemplaza valores anómalos en fechas específicas utilizando una estrategia híbrida.
    Adaptado para trabajar con lista de DataFrames con índice 'fecha'.
    
    Args:
        timeseries_list: Lista de DataFrames con índice 'fecha' y columna 'cantidad'
        anomalous_dates: Lista de fechas consideradas anómalas
        
    Returns:
        Tuple: (lista de series procesadas, diccionario con valores originales guardados)
    """
    # Convertir fechas anómalas a datetime
    anomalous_dates = [pd.to_datetime(date) for date in anomalous_dates]
    
    # Diccionario para almacenar valores originales (usando índice de la lista como clave)
    original_values = {}
    
    # Lista para almacenar series procesadas
    processed_series = []
    
    # Procesar cada DataFrame
    for i, df in enumerate(tqdm(timeseries_list, desc="Procesando fechas anómalas")):
        # Verificar que es un DataFrame válido
        if not isinstance(df, pd.DataFrame) or df.empty:
            processed_series.append(df)
            continue
        
        # Hacer una copia para no modificar el original
        df_copy = df.copy()
        
        # Verificar que el índice es 'fecha' o convertirlo
        if df_copy.index.name != 'fecha':
            if 'fecha' in df_copy.columns:
                df_copy.set_index('fecha', inplace=True)
            else:
                print(f"DataFrame {i}: No se encontró la columna o índice 'fecha'")
                processed_series.append(df)
                continue
        
        # Verificar que existe la columna 'cantidad'
        if 'cantidad' not in df_copy.columns:
            print(f"DataFrame {i}: No se encontró la columna 'cantidad'")
            processed_series.append(df)
            continue
        
        # Asegurarse de que el índice es datetime
        if not isinstance(df_copy.index, pd.DatetimeIndex):
            df_copy.index = pd.to_datetime(df_copy.index)
        
        # Inicializar diccionario para este DataFrame
        original_values[i] = {}
        
        # Para cada fecha anómala
        for anomalous_date in anomalous_dates:
            # Verificar si esta fecha existe en el DataFrame
            if anomalous_date not in df_copy.index:
                continue
            
            # Obtener el valor original
            if not pd.isna(df_copy.loc[anomalous_date, 'cantidad']):
                original_value = df_copy.loc[anomalous_date, 'cantidad']
                
                # Guardar el valor original
                original_values[i][anomalous_date.strftime('%Y-%m-%d')] = original_value
            else:
                # Si no hay dato para esta fecha o es NaN, continuar con la siguiente fecha
                continue
            
            # 1. Obtener el valor del mismo día del año anterior (V₁)
            previous_year_date = anomalous_date - pd.DateOffset(years=1)
            
            if previous_year_date in df_copy.index and not pd.isna(df_copy.loc[previous_year_date, 'cantidad']):
                v1 = df_copy.loc[previous_year_date, 'cantidad']
            else:
                # Si no hay valor del año anterior, v1 será None
                v1 = None
            
            # 2. Calcular la mediana del día de la semana correspondiente (V₂)
            weekday = anomalous_date.dayofweek
            
            # Crear máscara para el día de la semana, excluyendo fechas anómalas
            weekday_mask = (df_copy.index.dayofweek == weekday) & (~df_copy.index.isin(anomalous_dates))
            weekday_data = df_copy.loc[weekday_mask, 'cantidad'].dropna()
            
            if len(weekday_data) >= 3:  # Al menos 3 valores para una mediana confiable
                v2 = weekday_data.median()
            else:
                # Si no hay suficientes datos para este día, usar todos los datos no anómalos
                exclude_anomalous_mask = ~df_copy.index.isin(anomalous_dates)
                all_data = df_copy.loc[exclude_anomalous_mask, 'cantidad'].dropna()
                
                if len(all_data) >= 3:
                    v2 = all_data.median()
                else:
                    # Si no hay suficientes datos, mantener el valor original
                    continue
            
            # 3. Aplicar reglas de decisión
            if v1 is not None:
                # Calcular la diferencia relativa
                relative_diff = abs(v1 - v2) / v2 if v2 != 0 else float('inf')
                
                # Verificar si v1 es atípico (>2.5 veces la mediana o <0.4 veces)
                is_v1_outlier = (v1 > 2.5 * v2) or (v1 < 0.4 * v2)
                
                if not is_v1_outlier and relative_diff < 0.5:
                    # Si la diferencia es menor al 50% y v1 no es outlier, usar v1
                    replacement_value = v1
                    method = "valor_año_anterior"
                elif is_v1_outlier:
                    # Si v1 es claramente atípico, usar v2
                    replacement_value = v2
                    method = "mediana_dia_semana"
                else:
                    # Si la diferencia es mayor pero v1 no es atípico, calcular promedio ponderado
                    replacement_value = 0.6 * v1 + 0.4 * v2
                    method = "promedio_ponderado"
            else:
                # Si no hay valor del año anterior, usar v2
                replacement_value = v2
                method = "mediana_dia_semana"
            
            # Reemplazar el valor anómalo
            df_copy.loc[anomalous_date, 'cantidad'] = replacement_value
            
            # Preparar información sobre valores V1 y V2 para mostrar
            if v1 is not None:
                v1_info = f"{v1:.2f}"
            else:
                v1_info = "No disponible"
            
            # Mostrar info sobre el reemplazo
            print(f"DataFrame {i} - {anomalous_date.strftime('%Y-%m-%d')}: "
                  f"Original: {original_value:.2f} → Reemplazo: {replacement_value:.2f} "
                  f"(Método: {method}, V1: {v1_info}, V2: {v2:.2f})")
        
        # Guardar DataFrame procesado
        processed_series.append(df_copy)
    
    return processed_series, original_values

# Ejemplo de uso:
# processed_timeseries, original_vals = handle_anomalous_dates_list(timeseries)


In [46]:
timeseries, original_vals = handle_anomalous_dates_list(timeseries)

Procesando fechas anómalas:   0%|          | 0/27 [00:00<?, ?it/s]

DataFrame 0 - 2023-11-01: Original: 9.00 → Reemplazo: 1.00 (Método: mediana_dia_semana, V1: No disponible, V2: 1.00)
DataFrame 0 - 2023-11-02: Original: 24.00 → Reemplazo: 1.00 (Método: mediana_dia_semana, V1: No disponible, V2: 1.00)
DataFrame 1 - 2023-11-01: Original: 11.00 → Reemplazo: 1.00 (Método: mediana_dia_semana, V1: No disponible, V2: 1.00)
DataFrame 1 - 2023-11-02: Original: 9.00 → Reemplazo: 1.00 (Método: mediana_dia_semana, V1: No disponible, V2: 1.00)
DataFrame 2 - 2023-11-01: Original: 13.00 → Reemplazo: 1.00 (Método: mediana_dia_semana, V1: No disponible, V2: 1.00)
DataFrame 2 - 2023-11-02: Original: 12.00 → Reemplazo: 1.00 (Método: mediana_dia_semana, V1: No disponible, V2: 1.00)
DataFrame 3 - 2023-11-01: Original: 16.00 → Reemplazo: 1.00 (Método: mediana_dia_semana, V1: No disponible, V2: 1.00)
DataFrame 3 - 2023-11-02: Original: 15.00 → Reemplazo: 1.00 (Método: mediana_dia_semana, V1: No disponible, V2: 1.00)
DataFrame 4 - 2023-11-01: Original: 16.00 → Reemplazo: 1.0

Procesando fechas anómalas: 100%|██████████| 27/27 [00:00<00:00, 334.34it/s]

DataFrame 10 - 2023-11-01: Original: 50.00 → Reemplazo: 4.00 (Método: mediana_dia_semana, V1: No disponible, V2: 4.00)
DataFrame 10 - 2023-11-02: Original: 83.00 → Reemplazo: 4.00 (Método: mediana_dia_semana, V1: No disponible, V2: 4.00)
DataFrame 11 - 2023-11-01: Original: 12.00 → Reemplazo: 1.00 (Método: mediana_dia_semana, V1: No disponible, V2: 1.00)
DataFrame 11 - 2023-11-02: Original: 34.00 → Reemplazo: 1.00 (Método: mediana_dia_semana, V1: No disponible, V2: 1.00)
DataFrame 12 - 2023-11-01: Original: 26.00 → Reemplazo: 1.00 (Método: mediana_dia_semana, V1: No disponible, V2: 1.00)
DataFrame 12 - 2023-11-02: Original: 34.00 → Reemplazo: 2.00 (Método: mediana_dia_semana, V1: No disponible, V2: 2.00)
DataFrame 13 - 2023-11-01: Original: 13.00 → Reemplazo: 1.00 (Método: mediana_dia_semana, V1: No disponible, V2: 1.00)
DataFrame 13 - 2023-11-02: Original: 13.00 → Reemplazo: 1.00 (Método: mediana_dia_semana, V1: No disponible, V2: 1.00)
DataFrame 14 - 2023-11-01: Original: 107.00 → Re

In [47]:
def limpiar_y_segmentar_series_list(timeseries_list, umbral_gap=30, puntos_min=10):
    """
    Identifica grupos en series temporales basados en gaps y elimina grupos pequeños.
    Solo reincorpora los NaNs que caen dentro del rango de fechas de los datos filtrados.
    Adaptado para lista de DataFrames con índice 'fecha'.
    
    Args:
        timeseries_list: Lista de DataFrames con índice 'fecha' y columna 'cantidad'
        umbral_gap: Número de días que define un gap entre grupos
        puntos_min: Número mínimo de puntos que debe tener un grupo para conservarse
    
    Returns:
        Lista con series filtradas
    """
    resultado = []

    for i, df in enumerate(tqdm(timeseries_list, desc="Limpiando y segmentando series")):
        # Verificar que es un DataFrame válido
        if not isinstance(df, pd.DataFrame) or df.empty:
            resultado.append(df)
            continue
        
        df_copy = df.copy()
        
        # Verificar que el índice es 'fecha' o convertirlo
        if df_copy.index.name != 'fecha':
            if 'fecha' in df_copy.columns:
                df_copy.set_index('fecha', inplace=True)
            else:
                print(f"DataFrame {i}: No se encontró la columna o índice 'fecha'")
                resultado.append(df)
                continue
        
        # Verificar que existe la columna 'cantidad'
        if 'cantidad' not in df_copy.columns:
            print(f"DataFrame {i}: No se encontró la columna 'cantidad'")
            resultado.append(df)
            continue
        
        # Asegurarse de que el índice es datetime
        if not isinstance(df_copy.index, pd.DatetimeIndex):
            df_copy.index = pd.to_datetime(df_copy.index)
        
        # Resetear índice para trabajar con columnas (temporalmente)
        df_work = df_copy.reset_index()

        # Paso 1: eliminar registros con 'cantidad' NaN y guardar aparte
        df_nan = df_work[df_work['cantidad'].isna()]
        df_clean = df_work.dropna(subset=['cantidad']).sort_values('fecha')

        if df_clean.empty:
            # Solo NaNs, devolver como está (con índice restaurado)
            df_nan_final = df_nan.set_index('fecha')
            resultado.append(df_nan_final)
            continue

        # Paso 2 y 3: calcular diferencias entre fechas sucesivas
        df_clean['gap'] = df_clean['fecha'].diff().dt.days.fillna(0)

        # Paso 4: asignar grupos según umbral_gap
        df_clean['grupo'] = (df_clean['gap'] >= umbral_gap).cumsum()

        # Paso 5: filtrar grupos con menos de puntos_min registros
        grupos_validos = df_clean['grupo'].value_counts()
        grupos_validos = grupos_validos[grupos_validos >= puntos_min].index

        df_filtrado = df_clean[df_clean['grupo'].isin(grupos_validos)].drop(columns=['gap', 'grupo'])

        # MEJORA: Reincorporar solo los NaNs que caen dentro del rango de fechas de los datos filtrados
        if not df_filtrado.empty:
            # Obtener el rango de fechas de los datos filtrados
            fecha_min = df_filtrado['fecha'].min()
            fecha_max = df_filtrado['fecha'].max()
            
            # Filtrar los NaNs para incluir solo los que están dentro del rango
            df_nan_filtrado = df_nan[(df_nan['fecha'] >= fecha_min) & (df_nan['fecha'] <= fecha_max)]
            
            # Concatenar y ordenar
            df_final = pd.concat([df_filtrado, df_nan_filtrado]).sort_values('fecha')
        else:
            # Si no quedaron datos después del filtrado, devolver un DataFrame vacío
            df_final = pd.DataFrame(columns=df_work.columns)

        # Restaurar el índice 'fecha'
        if not df_final.empty:
            df_final = df_final.set_index('fecha')
        else:
            # DataFrame vacío con índice correcto
            df_final = pd.DataFrame(columns=df_copy.columns)
            df_final.index.name = 'fecha'

        resultado.append(df_final)

    return resultado

In [48]:
timeseries = limpiar_y_segmentar_series_list(
    timeseries, 
    umbral_gap=30,    # Días para considerar un gap
    puntos_min=10     # Mínimo de puntos por grupo
)


Limpiando y segmentando series: 100%|██████████| 27/27 [00:00<00:00, 181.43it/s]


In [49]:
freqs = check_frequencies(timeseries)

timeseries[0] → D
timeseries[1] → D
timeseries[2] → D
timeseries[3] → D
timeseries[4] → D
timeseries[5] → D
timeseries[6] → None
timeseries[7] → None
timeseries[8] → D
timeseries[9] → D
timeseries[10] → D
timeseries[11] → D
timeseries[12] → D
timeseries[13] → D
timeseries[14] → D
timeseries[15] → D
timeseries[16] → D
timeseries[17] → D
timeseries[18] → D
timeseries[19] → D
timeseries[20] → D
timeseries[21] → D
timeseries[22] → D
timeseries[23] → D
timeseries[24] → D
timeseries[25] → D
timeseries[26] → D


In [50]:
timeseries = completar_series_temporales_list(timeseries)

Completando series temporales: 100%|██████████| 27/27 [00:00<00:00, 235.19it/s]

DataFrame 0: 1370 → 1370 registros (0 fechas agregadas)
DataFrame 1: 1282 → 1282 registros (0 fechas agregadas)
DataFrame 2: 1374 → 1374 registros (0 fechas agregadas)
DataFrame 3: 1370 → 1370 registros (0 fechas agregadas)
DataFrame 4: 1372 → 1372 registros (0 fechas agregadas)
DataFrame 5: 1360 → 1360 registros (0 fechas agregadas)
DataFrame 6: 1367 → 1373 registros (6 fechas agregadas)
DataFrame 7: 1330 → 1339 registros (9 fechas agregadas)
DataFrame 8: 1198 → 1198 registros (0 fechas agregadas)
DataFrame 9: 1366 → 1366 registros (0 fechas agregadas)
DataFrame 10: 1373 → 1373 registros (0 fechas agregadas)
DataFrame 11: 1356 → 1356 registros (0 fechas agregadas)
DataFrame 12: 1366 → 1366 registros (0 fechas agregadas)
DataFrame 13: 1374 → 1374 registros (0 fechas agregadas)
DataFrame 14: 1374 → 1374 registros (0 fechas agregadas)
DataFrame 15: 1368 → 1368 registros (0 fechas agregadas)
DataFrame 16: 1349 → 1349 registros (0 fechas agregadas)
DataFrame 17: 1316 → 1316 registros (0 fe

In [51]:
freqs = check_frequencies(timeseries)

timeseries[0] → <Day>
timeseries[1] → <Day>
timeseries[2] → <Day>
timeseries[3] → <Day>
timeseries[4] → <Day>
timeseries[5] → <Day>
timeseries[6] → <Day>
timeseries[7] → <Day>
timeseries[8] → <Day>
timeseries[9] → <Day>
timeseries[10] → <Day>
timeseries[11] → <Day>
timeseries[12] → <Day>
timeseries[13] → <Day>
timeseries[14] → <Day>
timeseries[15] → <Day>
timeseries[16] → <Day>
timeseries[17] → <Day>
timeseries[18] → <Day>
timeseries[19] → <Day>
timeseries[20] → <Day>
timeseries[21] → <Day>
timeseries[22] → <Day>
timeseries[23] → <Day>
timeseries[24] → <Day>
timeseries[25] → <Day>
timeseries[26] → <Day>


In [52]:
resultados

{'dataframe_0': 33.0,
 'dataframe_1': 20.0,
 'dataframe_2': 25.0,
 'dataframe_3': 31.0,
 'dataframe_4': 28.0,
 'dataframe_5': 44.0,
 'dataframe_6': 4.0,
 'dataframe_7': 5.0,
 'dataframe_8': 41.0,
 'dataframe_9': 49.0,
 'dataframe_10': 133.0,
 'dataframe_11': 46.0,
 'dataframe_12': 60.0,
 'dataframe_13': 26.0,
 'dataframe_14': 254.0,
 'dataframe_15': 280.0,
 'dataframe_16': 50.0,
 'dataframe_17': 43.0,
 'dataframe_18': 46.0,
 'dataframe_19': 360.0,
 'dataframe_20': 77.0,
 'dataframe_21': 87.0,
 'dataframe_22': 100.0,
 'dataframe_23': 82.0,
 'dataframe_24': 43.0,
 'dataframe_25': 76.0,
 'dataframe_26': 486.0}

In [53]:
def redistribuir_valores_list(timeseries_list, resultados):
    """
    Redistribuye las sumas de 2023-11-01 y 2023-11-02 en el intervalo [2023-09-10, 2023-10-31],
    distribuyendo solo en puntos correspondientes al año anterior [2022-09-10, 2022-10-31],
    respetando el ratio del año anterior.
    
    Args:
        timeseries_list: Lista de DataFrames con índice 'fecha' y columna 'cantidad'
        resultados: Diccionario con sumas por DataFrame {'dataframe_0': suma_total, ...}
        
    Returns:
        Lista con DataFrames ajustados
    """
    resultado_ajustado = []
    
    # Fechas de outliers y períodos
    fecha_outlier1 = pd.Timestamp("2023-11-01")
    fecha_outlier2 = pd.Timestamp("2023-11-02")
    fecha_destino_inicio = pd.Timestamp("2023-09-10")
    fecha_destino_fin = pd.Timestamp("2023-10-31")
    fecha_base_inicio = pd.Timestamp("2022-09-10")
    fecha_base_fin = pd.Timestamp("2022-10-31")
    
    # Imprimir las claves disponibles para diagnóstico
    print("Claves disponibles en resultados:", list(resultados.keys()))
    
    for i, df in enumerate(tqdm(timeseries_list, desc="Redistribuyendo valores")):
        print(f"\n{'='*60}")
        print(f"Procesando DataFrame {i}")
        print(f"{'='*60}")
        
        # Verificar que es un DataFrame válido
        if not isinstance(df, pd.DataFrame) or df.empty:
            resultado_ajustado.append(df)
            continue
        
        df_copy = df.copy()
        
        # Verificar que el índice es 'fecha' o convertirlo
        if df_copy.index.name != 'fecha':
            if 'fecha' in df_copy.columns:
                df_copy.set_index('fecha', inplace=True)
            else:
                print(f"DataFrame {i}: No se encontró la columna o índice 'fecha'")
                resultado_ajustado.append(df)
                continue
        
        # Verificar columna 'cantidad'
        if 'cantidad' not in df_copy.columns:
            print(f"DataFrame {i}: No se encontró la columna 'cantidad'")
            resultado_ajustado.append(df)
            continue
        
        # Asegurar que el índice es datetime
        if not isinstance(df_copy.index, pd.DatetimeIndex):
            df_copy.index = pd.to_datetime(df_copy.index)
        
        # Obtener suma a redistribuir del diccionario resultados
        clave_df = f'dataframe_{i}'
        print(f"Buscando clave: {clave_df}")
        
        if clave_df not in resultados:
            print(f"DataFrame {i} ({clave_df}): No se encontró en el diccionario de resultados")
            resultado_ajustado.append(df_copy)
            continue
        
        suma_a_redistribuir = resultados[clave_df]
        print(f"Suma a redistribuir: {suma_a_redistribuir}")
        
        if suma_a_redistribuir <= 0:
            print("Suma negativa o cero, no hay necesidad de redistribuir")
            resultado_ajustado.append(df_copy)
            continue
        
        # Paso 1: Obtener datos del período base (2022)
        mask_base = (df_copy.index >= fecha_base_inicio) & (df_copy.index <= fecha_base_fin)
        df_base = df_copy.loc[mask_base].dropna(subset=['cantidad'])
        
        if df_base.empty:
            print("No hay datos válidos en el período base 2022, no se puede redistribuir")
            resultado_ajustado.append(df_copy)
            continue
        
        print(f"Días con valores en 2022: {len(df_base)}")
        
        # Paso 2: Crear mapeo de fechas entre 2022 y 2023
        fechas_2022 = pd.date_range(start=fecha_base_inicio, end=fecha_base_fin, freq='D')
        fechas_2023 = pd.date_range(start=fecha_destino_inicio, end=fecha_destino_fin, freq='D')
        
        if len(fechas_2022) != len(fechas_2023):
            print(f"Los períodos tienen diferente duración: 2022 ({len(fechas_2022)} días) vs 2023 ({len(fechas_2023)} días)")
            resultado_ajustado.append(df_copy)
            continue
        
        # Crear mapeo entre fechas
        mapa_2022_a_2023 = dict(zip(fechas_2022, fechas_2023))
        
        # Paso 3: Identificar fechas en 2023 que corresponden a valores existentes en 2022
        fechas_destino_correspondientes = []
        valores_base_correspondientes = []
        
        for fecha_2022 in df_base.index:
            if fecha_2022 in mapa_2022_a_2023:
                fecha_2023 = mapa_2022_a_2023[fecha_2022]
                fechas_destino_correspondientes.append(fecha_2023)
                valores_base_correspondientes.append(df_base.loc[fecha_2022, 'cantidad'])
        
        if not fechas_destino_correspondientes:
            print("No hay fechas correspondientes entre 2022 y 2023")
            resultado_ajustado.append(df_copy)
            continue
        
        # Paso 4: Filtrar fechas que no tienen valores en 2023
        mask_destino = (df_copy.index >= fecha_destino_inicio) & (df_copy.index <= fecha_destino_fin)
        fechas_con_valores_2023 = set(df_copy.loc[mask_destino].dropna(subset=['cantidad']).index)
        
        # Fechas disponibles para imputar (que corresponden a 2022 pero no tienen valor en 2023)
        fechas_disponibles = []
        valores_correspondientes = []
        
        for fecha_2023, valor_2022 in zip(fechas_destino_correspondientes, valores_base_correspondientes):
            if fecha_2023 not in fechas_con_valores_2023:
                fechas_disponibles.append(fecha_2023)
                valores_correspondientes.append(valor_2022)
        
        print(f"Fechas disponibles para imputar: {len(fechas_disponibles)}")
        
        if not fechas_disponibles:
            print("No hay fechas disponibles para imputar")
            resultado_ajustado.append(df_copy)
            continue
        
        # Paso 5: Si hay 20 o más puntos, seleccionar la mitad intercalados
        if len(fechas_disponibles) >= 20:
            # Calcular cuántos puntos seleccionar (la mitad)
            puntos_a_seleccionar = len(fechas_disponibles) // 2
            
            # Crear DataFrame para ordenar por valor
            df_disponibles = pd.DataFrame({
                'fecha': fechas_disponibles,
                'valor_2022': valores_correspondientes
            }).sort_values('valor_2022', ascending=False)
            
            # Calcular el paso para intercalar
            total_disponibles = len(df_disponibles)
            paso = max(1, total_disponibles // puntos_a_seleccionar)
            
            # Tomar puntos intercalados
            indices_seleccionados = list(range(0, total_disponibles, paso))[:puntos_a_seleccionar]
            df_seleccionados = df_disponibles.iloc[indices_seleccionados]
            
            fechas_a_imputar = df_seleccionados['fecha'].tolist()
            valores_base_para_ratio = df_seleccionados['valor_2022'].tolist()
            
            print(f"Seleccionados {len(fechas_a_imputar)} puntos intercalados (mitad) de {len(fechas_disponibles)} disponibles")
        else:
            fechas_a_imputar = fechas_disponibles
            valores_base_para_ratio = valores_correspondientes
            print(f"Usando todos los {len(fechas_a_imputar)} puntos disponibles")
        
        # Paso 6: Calcular ratios basados en valores de 2022
        suma_base = sum(valores_base_para_ratio)
        if suma_base == 0:
            print("Suma de valores base es cero, usando distribución uniforme")
            ratios = [1.0 / len(fechas_a_imputar)] * len(fechas_a_imputar)
        else:
            ratios = [valor / suma_base for valor in valores_base_para_ratio]
        
        # Paso 7: Calcular valores a imputar
        valores_a_imputar = [ratio * suma_a_redistribuir for ratio in ratios]
        
        print(f"Valores a imputar - Min: {min(valores_a_imputar):.2f}, Max: {max(valores_a_imputar):.2f}, Total: {sum(valores_a_imputar):.2f}")
        
        # Paso 8: Aplicar los valores al DataFrame
        df_final = df_copy.copy()
        
        for fecha, valor in zip(fechas_a_imputar, valores_a_imputar):
            if fecha in df_final.index:
                df_final.loc[fecha, 'cantidad'] = valor
            else:
                # Crear nueva fila con las mismas columnas que el DataFrame original
                nueva_fila = pd.Series(index=df_final.columns, dtype=object)
                nueva_fila['cantidad'] = valor
                
                # Rellenar otras columnas con valores típicos del DataFrame
                for col in df_final.columns:
                    if col != 'cantidad':
                        valores_no_nulos = df_final[col].dropna()
                        if not valores_no_nulos.empty:
                            if valores_no_nulos.dtype in ['object', 'string']:
                                nueva_fila[col] = valores_no_nulos.mode().iloc[0] if len(valores_no_nulos.mode()) > 0 else valores_no_nulos.iloc[0]
                            else:
                                moda = valores_no_nulos.mode()
                                nueva_fila[col] = moda.iloc[0] if len(moda) > 0 else valores_no_nulos.median()
                
                # Agregar la nueva fila
                nueva_fila.name = fecha
                df_final = pd.concat([df_final, nueva_fila.to_frame().T])
        
        # Ordenar por índice (fecha)
        df_final = df_final.sort_index()
        
        # Verificación final
        suma_destino_final = df_final.loc[mask_destino, 'cantidad'].sum()
        suma_outliers_final = 0
        if fecha_outlier1 in df_final.index:
            suma_outliers_final += df_final.loc[fecha_outlier1, 'cantidad']
        if fecha_outlier2 in df_final.index:
            suma_outliers_final += df_final.loc[fecha_outlier2, 'cantidad']
        
        print(f"Verificación final:")
        print(f"Suma en período destino: {suma_destino_final:.2f}")
        print(f"Suma en outliers: {suma_outliers_final:.2f}")
        print(f"Total redistribuido: {suma_destino_final:.2f}")
        print(f"Diferencia con objetivo: {suma_destino_final - suma_a_redistribuir:.2f}")
        
        resultado_ajustado.append(df_final)
    
    return resultado_ajustado

In [54]:
timeseries = redistribuir_valores_list(timeseries, resultados)

Claves disponibles en resultados: ['dataframe_0', 'dataframe_1', 'dataframe_2', 'dataframe_3', 'dataframe_4', 'dataframe_5', 'dataframe_6', 'dataframe_7', 'dataframe_8', 'dataframe_9', 'dataframe_10', 'dataframe_11', 'dataframe_12', 'dataframe_13', 'dataframe_14', 'dataframe_15', 'dataframe_16', 'dataframe_17', 'dataframe_18', 'dataframe_19', 'dataframe_20', 'dataframe_21', 'dataframe_22', 'dataframe_23', 'dataframe_24', 'dataframe_25', 'dataframe_26']


Redistribuyendo valores:   0%|          | 0/27 [00:00<?, ?it/s]


Procesando DataFrame 0
Buscando clave: dataframe_0
Suma a redistribuir: 33.0
Días con valores en 2022: 14
Fechas disponibles para imputar: 14
Usando todos los 14 puntos disponibles
Valores a imputar - Min: 1.83, Max: 7.33, Total: 33.00
Verificación final:
Suma en período destino: 33.00
Suma en outliers: 2.00
Total redistribuido: 33.00
Diferencia con objetivo: 0.00

Procesando DataFrame 1
Buscando clave: dataframe_1
Suma a redistribuir: 20.0
Días con valores en 2022: 17
Fechas disponibles para imputar: 16
Usando todos los 16 puntos disponibles
Valores a imputar - Min: 0.67, Max: 2.67, Total: 20.00
Verificación final:
Suma en período destino: 21.00
Suma en outliers: 2.00
Total redistribuido: 21.00
Diferencia con objetivo: 1.00

Procesando DataFrame 2
Buscando clave: dataframe_2
Suma a redistribuir: 25.0
Días con valores en 2022: 27
Fechas disponibles para imputar: 26
Seleccionados 13 puntos intercalados (mitad) de 26 disponibles
Valores a imputar - Min: 0.93, Max: 5.56, Total: 25.00
Ver

Redistribuyendo valores: 100%|██████████| 27/27 [00:00<00:00, 136.18it/s]

Verificación final:
Suma en período destino: 33.00
Suma en outliers: 2.00
Total redistribuido: 33.00
Diferencia con objetivo: 2.00

Procesando DataFrame 4
Buscando clave: dataframe_4
Suma a redistribuir: 28.0
Días con valores en 2022: 21
Fechas disponibles para imputar: 20
Seleccionados 10 puntos intercalados (mitad) de 20 disponibles
Valores a imputar - Min: 1.33, Max: 8.00, Total: 28.00
Verificación final:
Suma en período destino: 30.00
Suma en outliers: 2.00
Total redistribuido: 30.00
Diferencia con objetivo: 2.00

Procesando DataFrame 5
Buscando clave: dataframe_5
Suma a redistribuir: 44.0
Días con valores en 2022: 33
Fechas disponibles para imputar: 33
Seleccionados 16 puntos intercalados (mitad) de 33 disponibles
Valores a imputar - Min: 1.13, Max: 6.77, Total: 44.00
Verificación final:
Suma en período destino: 46.00
Suma en outliers: 2.00
Total redistribuido: 46.00
Diferencia con objetivo: 2.00

Procesando DataFrame 6
Buscando clave: dataframe_6
Suma a redistribuir: 4.0
Días con

In [55]:
def codificar_columnas_categoricas(lista_dataframes):
    """
    Codifica las columnas categóricas usando códigos numéricos únicos para cada valor.
    Las columnas a codificar son: Tipo_Producto, segmento_producto, supergrupo_producto, 
    grupo_producto y subgrupo_producto.
    
    Args:
        lista_dataframes: Lista de dataframes con columnas categóricas
    
    Returns:
        Lista de dataframes con columnas categóricas codificadas y diccionario de mapeos
    """
    import pandas as pd
    
    # Columnas categóricas a codificar
    columnas_categoricas = [
        'Tipo_Producto', 
        'segmento_producto', 
        'supergrupo_producto', 
        'grupo_producto', 
        'subgrupo_producto'
    ]
    
    # Diccionario para almacenar los mapeos para cada columna
    mapeos = {col: {} for col in columnas_categoricas}
    
    # Lista para almacenar los dataframes codificados
    dataframes_codificados = []
    
    # Crear mapeos para cada columna categórica
    codigo_actual = {}
    for col in columnas_categoricas:
        codigo_actual[col] = 0
    
    # Iterar sobre cada dataframe para crear los mapeos
    for df in lista_dataframes:
        for col in columnas_categoricas:
            if col in df.columns:
                # Obtener el valor único en esta columna para este dataframe
                # (asumiendo que cada dataframe tiene un solo valor para cada columna categórica)
                if len(df) > 0:
                    valor = df[col].iloc[0]
                    
                    # Si el valor no está en el mapeo, asignarle un código
                    if valor not in mapeos[col]:
                        mapeos[col][valor] = codigo_actual[col]
                        codigo_actual[col] += 1
    
    # Aplicar los mapeos a cada dataframe
    for df in lista_dataframes:
        df_codificado = df.copy()
        
        for col in columnas_categoricas:
            if col in df.columns:
                if len(df) > 0:
                    valor = df[col].iloc[0]
                    codigo = mapeos[col][valor]
                    
                    # Reemplazar el valor categórico con su código
                    df_codificado[col] = codigo
        
        dataframes_codificados.append(df_codificado)
    
    return dataframes_codificados, mapeos

In [56]:
timeseries,mapeos = codificar_columnas_categoricas(timeseries)

In [57]:
def agregar_columnas_temporales_con_vectores(lista_dataframes):
    """
    Agrega columnas temporales y vector dinámico V1 a cada dataframe.
    
    Args:
        lista_dataframes: Lista de dataframes con índice de fecha
    
    Returns:
        Lista de dataframes con las nuevas columnas temporales y vector V1
        
    Columnas agregadas:
        V1: Vector de 0s excepto en fechas 2023-09-10 hasta 2023-11-02 (valor = 1)
        day: Día del mes (1-31)
        weekday: Día de la semana (0=lunes, 6=domingo)
        week: Número de semana del año (1-53)
        month: Mes (1-12)
        quarter: Trimestre (1-4)
    """
    resultado = []
    
    # Definir el rango de fechas especial para V1
    fecha_inicio_v1 = pd.Timestamp("2023-09-10")
    fecha_fin_v1 = pd.Timestamp("2023-11-02")
    
    for i, df in enumerate(lista_dataframes):
        print(f"Procesando DataFrame {i}...")
        
        # Verificar que es un DataFrame válido
        if not isinstance(df, pd.DataFrame) or df.empty:
            resultado.append(df)
            continue
        
        # Crear una copia del dataframe
        df_nuevo = df.copy()
        
        # Asegurarse de que el índice es de tipo datetime
        if not isinstance(df_nuevo.index, pd.DatetimeIndex):
            df_nuevo.index = pd.to_datetime(df_nuevo.index)
        
        # V1: Vector binario para el rango 2023-09-10 hasta 2023-11-02
        df_nuevo['V1'] = 0  # Inicializar todo con 0
        mask_v1 = (df_nuevo.index >= fecha_inicio_v1) & (df_nuevo.index <= fecha_fin_v1)
        df_nuevo.loc[mask_v1, 'V1'] = 1
        
        # Columnas temporales estándar
        df_nuevo['day'] = df_nuevo.index.day
        df_nuevo['weekday'] = df_nuevo.index.dayofweek
        df_nuevo['week'] = df_nuevo.index.isocalendar().week
        df_nuevo['month'] = df_nuevo.index.month
        df_nuevo['quarter'] = df_nuevo.index.quarter
        
        # Mostrar información sobre V1
        count_v1_ones = df_nuevo['V1'].sum()
        print(f"  DataFrame {i}: V1 tiene {count_v1_ones} fechas marcadas (2023-09-10 a 2023-11-02)")
        
        # Añadir a la lista de resultados
        resultado.append(df_nuevo)
    
    return resultado

def mostrar_info_vectores(lista_dataframes_con_vectores, indices_muestra=[0]):
    """
    Muestra información detallada sobre las columnas temporales creadas
    
    Args:
        lista_dataframes_con_vectores: Lista de DataFrames con columnas temporales
        indices_muestra: Lista de índices de DataFrames para mostrar como muestra
    """
    print("\n" + "="*80)
    print("INFORMACIÓN DE COLUMNAS TEMPORALES")
    print("="*80)
    
    for idx in indices_muestra:
        if idx < len(lista_dataframes_con_vectores):
            df = lista_dataframes_con_vectores[idx]
            
            if isinstance(df, pd.DataFrame) and not df.empty:
                print(f"\nDataFrame {idx}:")
                print(f"Rango de fechas: {df.index.min()} a {df.index.max()}")
                print(f"Total de registros: {len(df)}")
                
                # Información de V1
                v1_count = df['V1'].sum()
                fechas_v1 = df[df['V1'] == 1].index
                print(f"\nV1 (período especial 2023-09-10 a 2023-11-02):")
                print(f"  Fechas marcadas: {v1_count}")
                if len(fechas_v1) > 0:
                    print(f"  Primera fecha: {fechas_v1.min()}")
                    print(f"  Última fecha: {fechas_v1.max()}")
                
                # Rangos de otras columnas
                print(f"\nRangos de columnas temporales:")
                print(f"  day: {df['day'].min()} - {df['day'].max()}")
                print(f"  weekday: {df['weekday'].min()} - {df['weekday'].max()} (0=lun, 6=dom)")
                print(f"  week: {df['week'].min()} - {df['week'].max()}")
                print(f"  month: {df['month'].min()} - {df['month'].max()}")
                print(f"  quarter: {df['quarter'].min()} - {df['quarter'].max()}")
                
                # Muestra de los primeros registros
                print(f"\nMuestra de registros (primeros 3):")
                columnas_temporales = ['V1', 'day', 'weekday', 'week', 'month', 'quarter']
                if all(col in df.columns for col in columnas_temporales):
                    print(df[columnas_temporales].head(3))

def verificar_periodo_v1(lista_dataframes_con_vectores):
    """
    Verifica específicamente el vector V1 en el período de interés
    
    Args:
        lista_dataframes_con_vectores: Lista de DataFrames con V1
    """
    fecha_inicio = pd.Timestamp("2023-09-10")
    fecha_fin = pd.Timestamp("2023-11-02")
    
    print("\n" + "="*80)
    print("VERIFICACIÓN DEL VECTOR V1 (PERÍODO 2023-09-10 a 2023-11-02)")
    print("="*80)
    
    for i, df in enumerate(lista_dataframes_con_vectores[:3]):  # Solo los primeros 3 para no saturar
        if isinstance(df, pd.DataFrame) and not df.empty and 'V1' in df.columns:
            # Filtrar el período de interés
            mask_periodo = (df.index >= fecha_inicio) & (df.index <= fecha_fin)
            df_periodo = df.loc[mask_periodo]
            
            if not df_periodo.empty:
                print(f"\nDataFrame {i} - Período 2023-09-10 a 2023-11-02:")
                print(f"  Registros en el período: {len(df_periodo)}")
                print(f"  V1 = 1: {df_periodo['V1'].sum()}")
                print(f"  V1 = 0: {(df_periodo['V1'] == 0).sum()}")
                
                # Mostrar algunas fechas específicas
                fechas_ejemplo = [
                    pd.Timestamp("2023-09-10"),
                    pd.Timestamp("2023-10-15"),
                    pd.Timestamp("2023-11-01"),
                    pd.Timestamp("2023-11-02")
                ]
                
                print("  Valores de V1 en fechas específicas:")
                for fecha in fechas_ejemplo:
                    if fecha in df.index:
                        valor_v1 = df.loc[fecha, 'V1']
                        print(f"    {fecha.date()}: V1 = {valor_v1}")

# Ejemplo de uso:
# dataframes_con_vectores = agregar_columnas_temporales_con_vectores(lista_dataframes)
# mostrar_info_vectores(dataframes_con_vectores, indices_muestra=[0, 1, 2])
# verificar_periodo_v1(dataframes_con_vectores)

In [58]:
timeseries = agregar_columnas_temporales_con_vectores(timeseries)
mostrar_info_vectores(timeseries, indices_muestra=[0, 1, 2])
verificar_periodo_v1(timeseries)

Procesando DataFrame 0...
  DataFrame 0: V1 tiene 54 fechas marcadas (2023-09-10 a 2023-11-02)
Procesando DataFrame 1...
  DataFrame 1: V1 tiene 54 fechas marcadas (2023-09-10 a 2023-11-02)
Procesando DataFrame 2...
  DataFrame 2: V1 tiene 54 fechas marcadas (2023-09-10 a 2023-11-02)
Procesando DataFrame 3...
  DataFrame 3: V1 tiene 54 fechas marcadas (2023-09-10 a 2023-11-02)
Procesando DataFrame 4...
  DataFrame 4: V1 tiene 54 fechas marcadas (2023-09-10 a 2023-11-02)
Procesando DataFrame 5...
  DataFrame 5: V1 tiene 54 fechas marcadas (2023-09-10 a 2023-11-02)
Procesando DataFrame 6...
  DataFrame 6: V1 tiene 54 fechas marcadas (2023-09-10 a 2023-11-02)
Procesando DataFrame 7...
  DataFrame 7: V1 tiene 54 fechas marcadas (2023-09-10 a 2023-11-02)
Procesando DataFrame 8...
  DataFrame 8: V1 tiene 54 fechas marcadas (2023-09-10 a 2023-11-02)
Procesando DataFrame 9...
  DataFrame 9: V1 tiene 54 fechas marcadas (2023-09-10 a 2023-11-02)
Procesando DataFrame 10...
  DataFrame 10: V1 tien

In [59]:
def aproximar_cantidad_a_enteros(timeseries_list):
    """
    Aproxima la columna 'cantidad' al entero más cercano en cada DataFrame.
    Si el valor es 0 o menor, se aproxima a 1.
    
    Args:
        timeseries_list: Lista de DataFrames con columna 'cantidad'
    
    Returns:
        Lista de DataFrames con la columna 'cantidad' aproximada a enteros
    """
    dataframes_modificados = []
    
    print(f"Procesando {len(timeseries_list)} DataFrames para aproximar 'cantidad' a enteros...")
    
    for i, df in enumerate(tqdm(timeseries_list, desc="Aproximando cantidades")):
        # Verificar que es un DataFrame válido
        if not isinstance(df, pd.DataFrame) or df.empty:
            print(f"Advertencia: DataFrame {i} está vacío o no es válido. Se mantiene sin cambios.")
            dataframes_modificados.append(df)
            continue
        
        # Crear copia del DataFrame
        df_modificado = df.copy()
        
        # Verificar que existe la columna 'cantidad'
        if 'cantidad' not in df_modificado.columns:
            print(f"Advertencia: DataFrame {i} no tiene columna 'cantidad'. Se mantiene sin cambios.")
            dataframes_modificados.append(df_modificado)
            continue
        
        # Obtener estadísticas antes de la modificación
        cantidad_original = df_modificado['cantidad'].copy()
        valores_nan = cantidad_original.isna().sum()
        valores_originales = cantidad_original.dropna()
        
        if len(valores_originales) == 0:
            print(f"Advertencia: DataFrame {i} no tiene valores válidos en 'cantidad'. Se mantiene sin cambios.")
            dataframes_modificados.append(df_modificado)
            continue
        
        # Estadísticas originales
        min_original = valores_originales.min()
        max_original = valores_originales.max()
        promedio_original = valores_originales.mean()
        ceros_negativos_original = (valores_originales <= 0).sum()
        
        # Aplicar aproximación
        # Paso 1: Redondear al entero más cercano
        cantidad_redondeada = np.round(cantidad_original)
        
        # Paso 2: Convertir valores <= 0 a 1
        cantidad_redondeada = np.where(cantidad_redondeada <= 0, 1, cantidad_redondeada)
        
        # Paso 3: Convertir a enteros (manteniendo NaN como NaN)
        mask_no_nan = ~cantidad_original.isna()
        df_modificado.loc[mask_no_nan, 'cantidad'] = cantidad_redondeada[mask_no_nan].astype(int)
        
        # Estadísticas después de la modificación
        cantidad_nueva = df_modificado['cantidad'].dropna()
        min_nuevo = cantidad_nueva.min()
        max_nuevo = cantidad_nueva.max()
        promedio_nuevo = cantidad_nueva.mean()
        valores_cambiados_a_1 = (cantidad_redondeada[mask_no_nan] == 1).sum() - (valores_originales == 1).sum()
        
        # Mostrar estadísticas de cambio (solo para los primeros DataFrames para no saturar)
        if i < 5:
            print(f"\nDataFrame {i} - Estadísticas de aproximación:")
            print(f"  Valores NaN: {valores_nan}")
            print(f"  Rango original: [{min_original:.2f}, {max_original:.2f}] -> Nuevo: [{min_nuevo}, {max_nuevo}]")
            print(f"  Promedio: {promedio_original:.2f} -> {promedio_nuevo:.2f}")
            print(f"  Valores ≤ 0 originales: {ceros_negativos_original}")
            print(f"  Valores cambiados a 1: {valores_cambiados_a_1}")
        
        dataframes_modificados.append(df_modificado)
    
    print(f"\nProcesamiento completado. {len(dataframes_modificados)} DataFrames procesados.")
    return dataframes_modificados

In [60]:
timeseries = aproximar_cantidad_a_enteros(timeseries)

Procesando 27 DataFrames para aproximar 'cantidad' a enteros...


Aproximando cantidades: 100%|██████████| 27/27 [00:00<00:00, 660.44it/s]


DataFrame 0 - Estadísticas de aproximación:
  Valores NaN: 840
  Rango original: [1.00, 7.33] -> Nuevo: [1.0, 7.0]
  Promedio: 1.70 -> 1.71
  Valores ≤ 0 originales: 0
  Valores cambiados a 1: 0

DataFrame 1 - Estadísticas de aproximación:
  Valores NaN: 951
  Rango original: [0.67, 5.00] -> Nuevo: [1.0, 5.0]
  Promedio: 1.47 -> 1.47
  Valores ≤ 0 originales: 0
  Valores cambiados a 1: 12

DataFrame 2 - Estadísticas de aproximación:
  Valores NaN: 822
  Rango original: [0.93, 8.00] -> Nuevo: [1.0, 8.0]
  Promedio: 1.88 -> 1.89
  Valores ≤ 0 originales: 0
  Valores cambiados a 1: 6

DataFrame 3 - Estadísticas de aproximación:
  Valores NaN: 828
  Rango original: [1.00, 7.75] -> Nuevo: [1.0, 8.0]
  Promedio: 1.76 -> 1.76
  Valores ≤ 0 originales: 0
  Valores cambiados a 1: 0

DataFrame 4 - Estadísticas de aproximación:
  Valores NaN: 972
  Rango original: [1.00, 8.00] -> Nuevo: [1.0, 8.0]
  Promedio: 1.82 -> 1.81
  Valores ≤ 0 originales: 0
  Valores cambiados a 1: 6

Procesamiento comp

In [61]:
freqs = check_frequencies(timeseries)

timeseries[0] → <Day>
timeseries[1] → <Day>
timeseries[2] → <Day>
timeseries[3] → <Day>
timeseries[4] → <Day>
timeseries[5] → <Day>
timeseries[6] → <Day>
timeseries[7] → <Day>
timeseries[8] → <Day>
timeseries[9] → <Day>
timeseries[10] → <Day>
timeseries[11] → <Day>
timeseries[12] → <Day>
timeseries[13] → <Day>
timeseries[14] → <Day>
timeseries[15] → <Day>
timeseries[16] → <Day>
timeseries[17] → <Day>
timeseries[18] → <Day>
timeseries[19] → <Day>
timeseries[20] → <Day>
timeseries[21] → <Day>
timeseries[22] → <Day>
timeseries[23] → <Day>
timeseries[24] → <Day>
timeseries[25] → <Day>
timeseries[26] → <Day>


In [62]:
def extraer_vectores_categoricos(lista_dataframes):
    """
    Extrae los valores categóricos del primer registro de cada dataframe y crea
    un vector con estos valores.
    
    Args:
        lista_dataframes: Lista de dataframes que contienen columnas categóricas
    
    Returns:
        Lista de vectores categóricos, uno por cada dataframe
    """
    # Definir las columnas categóricas
    columnas_categoricas = [
        'Tipo_Producto', 
        'segmento_producto', 
        'supergrupo_producto', 
        'grupo_producto', 
        'subgrupo_producto'
    ]
    
    vectores_categoricos = []
    
    for i, df in enumerate(lista_dataframes):
        # Verificar que el dataframe tenga registros
        if len(df) == 0:
            print(f"Advertencia: Dataframe {i} está vacío, se usará un vector de ceros.")
            vectores_categoricos.append([0] * len(columnas_categoricas))
            continue
        
        # Crear el vector para este dataframe
        vector = []
        
        for col in columnas_categoricas:
            # Verificar si la columna existe en el dataframe
            if col in df.columns:
                # Obtener el valor del primer registro
                valor = df[col].iloc[0]
                
                # Convertir a entero si es posible, de lo contrario usar un código hash
                try:
                    valor_numerico = int(valor)
                except (ValueError, TypeError):
                    # Si no se puede convertir a entero, usar un código hash simple
                    if valor is None:
                        valor_numerico = 0
                    else:
                        # Hash simple basado en la representación string del valor
                        valor_numerico = hash(str(valor)) % 10000  # Limitar a 4 dígitos
            else:
                # Si la columna no existe, usar 0
                valor_numerico = 0
            
            vector.append(valor_numerico)
        
        vectores_categoricos.append(vector)
        
    return vectores_categoricos

In [63]:
vectores_cat = extraer_vectores_categoricos(timeseries)

In [64]:
def extraer_vectores_cantidad(lista_dataframes):
    """
    Extrae los valores de la columna 'cantidad' de cada dataframe y crea 
    un vector con estos valores.
    
    Args:
        lista_dataframes: Lista de dataframes que contienen la columna 'cantidad'
    
    Returns:
        Lista de vectores, donde cada vector contiene los valores de 'cantidad' de un dataframe
    """
    import pandas as pd
    import numpy as np
    
    vectores_cantidad = []
    
    for i, df in enumerate(lista_dataframes):
        # Verificar que el dataframe tenga registros
        if len(df) == 0:
            print(f"Advertencia: Dataframe {i} está vacío.")
            vectores_cantidad.append([])
            continue
        
        # Verificar que exista la columna 'cantidad'
        if 'cantidad' not in df.columns:
            print(f"Advertencia: Dataframe {i} no tiene columna 'cantidad'.")
            vectores_cantidad.append([])
            continue
        
        # Extraer los valores de 'cantidad' como una lista
        valores = df['cantidad'].tolist()
        
        # Opcionalmente, puedes manejar valores NaN
        # valores = [0 if pd.isna(x) else x for x in valores]  # Convierte NaN a 0
        # O simplemente:
        valores = df['cantidad'].fillna(0).tolist()  # Rellena NaN con 0
        
        vectores_cantidad.append(valores)
    
    return vectores_cantidad

In [65]:
vectores_target = extraer_vectores_cantidad(timeseries)

In [66]:
def extraer_vectores_temporales(lista_dataframes):
    """
    Extrae los valores de las columnas temporales de cada dataframe
    y crea un conjunto de vectores con estos valores.
    
    Args:
        lista_dataframes: Lista de dataframes que contienen columnas temporales
    
    Returns:
        Lista de conjuntos de vectores, donde cada conjunto contiene los vectores
        de V1, day, weekday, week, month, quarter para un dataframe
    """
    conjuntos_vectores = []
    
    # Columnas temporales esperadas
    columnas_esperadas = ['V1', 'day', 'weekday', 'week', 'month', 'quarter']
    
    for i, df in enumerate(lista_dataframes):
        # Verificar que el dataframe tenga registros
        if len(df) == 0:
            print(f"Advertencia: Dataframe {i} está vacío.")
            # Crear vectores vacíos para todas las columnas
            vectores_vacios = tuple([] for _ in columnas_esperadas)
            conjuntos_vectores.append(vectores_vacios)
            continue
        
        # Verificar que existan las columnas necesarias
        columnas_faltantes = []
        for col in columnas_esperadas:
            if col not in df.columns:
                columnas_faltantes.append(col)
        
        if columnas_faltantes:
            print(f"Advertencia: Dataframe {i} no tiene las columnas: {columnas_faltantes}")
            
            # Si faltan las columnas, podemos crearlas a partir del índice si es de tipo fecha
            df_temp = df.copy()
            
            # Verificar si el índice es de tipo fecha
            if isinstance(df_temp.index, pd.DatetimeIndex):
                # Crear columnas temporales faltantes
                if 'day' not in df_temp.columns:
                    df_temp['day'] = df_temp.index.day
                if 'weekday' not in df_temp.columns:
                    df_temp['weekday'] = df_temp.index.dayofweek
                if 'week' not in df_temp.columns:
                    df_temp['week'] = df_temp.index.isocalendar().week
                if 'month' not in df_temp.columns:
                    df_temp['month'] = df_temp.index.month
                if 'quarter' not in df_temp.columns:
                    df_temp['quarter'] = df_temp.index.quarter
                if 'V1' not in df_temp.columns:
                    # Crear V1 basado en el período 2023-09-10 a 2023-11-02
                    fecha_inicio_v1 = pd.Timestamp("2023-09-10")
                    fecha_fin_v1 = pd.Timestamp("2023-11-02")
                    df_temp['V1'] = 0
                    mask_v1 = (df_temp.index >= fecha_inicio_v1) & (df_temp.index <= fecha_fin_v1)
                    df_temp.loc[mask_v1, 'V1'] = 1
            else:
                # Si el índice no es de tipo fecha, tratar de convertirlo
                try:
                    df_temp.index = pd.to_datetime(df_temp.index)
                    # Crear columnas temporales después de convertir el índice
                    if 'day' not in df_temp.columns:
                        df_temp['day'] = df_temp.index.day
                    if 'weekday' not in df_temp.columns:
                        df_temp['weekday'] = df_temp.index.dayofweek
                    if 'week' not in df_temp.columns:
                        df_temp['week'] = df_temp.index.isocalendar().week
                    if 'month' not in df_temp.columns:
                        df_temp['month'] = df_temp.index.month
                    if 'quarter' not in df_temp.columns:
                        df_temp['quarter'] = df_temp.index.quarter
                    if 'V1' not in df_temp.columns:
                        fecha_inicio_v1 = pd.Timestamp("2023-09-10")
                        fecha_fin_v1 = pd.Timestamp("2023-11-02")
                        df_temp['V1'] = 0
                        mask_v1 = (df_temp.index >= fecha_inicio_v1) & (df_temp.index <= fecha_fin_v1)
                        df_temp.loc[mask_v1, 'V1'] = 1
                except:
                    # Si no se puede convertir, usar vectores vacíos
                    print(f"Error: No se pudo convertir el índice del dataframe {i} a fecha")
                    vectores_vacios = tuple([] for _ in columnas_esperadas)
                    conjuntos_vectores.append(vectores_vacios)
                    continue
            
            # Usar el dataframe temporal con las columnas agregadas
            df = df_temp
        
        # Extraer los vectores en el orden especificado
        V1_vector = df['V1'].tolist()
        day_vector = df['day'].tolist()
        weekday_vector = df['weekday'].tolist()
        week_vector = df['week'].tolist()
        month_vector = df['month'].tolist()
        quarter_vector = df['quarter'].tolist()
        
        # Guardar el conjunto de vectores como tupla
        conjunto_vectores = (V1_vector, day_vector, weekday_vector, week_vector, month_vector, quarter_vector)
        conjuntos_vectores.append(conjunto_vectores)
        
        # Información de diagnóstico
        print(f"Dataframe {i}: Extraídos {len(V1_vector)} registros")
        print(f"  V1 valores únicos: {len(set(V1_vector))} - rango: {min(V1_vector)} a {max(V1_vector)}")
        print(f"  day rango: {min(day_vector)} a {max(day_vector)}")
        print(f"  weekday rango: {min(weekday_vector)} a {max(weekday_vector)}")
        print(f"  week rango: {min(week_vector)} a {max(week_vector)}")
        print(f"  month rango: {min(month_vector)} a {max(month_vector)}")
        print(f"  quarter rango: {min(quarter_vector)} a {max(quarter_vector)}")
    
    return conjuntos_vectores

def mostrar_resumen_vectores(conjuntos_vectores, indices_muestra=[0, 1, 2]):
    """
    Muestra un resumen de los vectores extraídos
    
    Args:
        conjuntos_vectores: Lista de tuplas con vectores temporales
        indices_muestra: Índices de dataframes para mostrar como muestra
    """
    print("\n" + "="*70)
    print("RESUMEN DE VECTORES EXTRAÍDOS")
    print("="*70)
    
    nombres_vectores = ['V1', 'day', 'weekday', 'week', 'month', 'quarter']
    
    for idx in indices_muestra:
        if idx < len(conjuntos_vectores):
            vectores = conjuntos_vectores[idx]
            
            if len(vectores) == 6:  # Verificar que tenemos todos los vectores
                print(f"\nDataframe {idx}:")
                print(f"Número de elementos: {len(vectores[0])}")
                
                for i, (nombre, vector) in enumerate(zip(nombres_vectores, vectores)):
                    if vector:  # Si el vector no está vacío
                        valores_unicos = len(set(vector))
                        rango_min, rango_max = min(vector), max(vector)
                        print(f"  {nombre}: {valores_unicos} valores únicos, rango [{rango_min}, {rango_max}]")
                        
                        # Para V1, mostrar cuántos 1s y 0s hay
                        if nombre == 'V1':
                            count_ones = vector.count(1)
                            count_zeros = vector.count(0)
                            print(f"    V1 detalles: {count_ones} unos, {count_zeros} ceros")
                    else:
                        print(f"  {nombre}: vector vacío")
            else:
                print(f"\nDataframe {idx}: Estructura de vectores incompleta")


# Ejemplo de uso:
# vectores_temporales = extraer_vectores_temporales(lista_dataframes)
# mostrar_resumen_vectores(vectores_temporales, indices_muestra=[0, 1, 2])

In [67]:
vectores_dynamic = extraer_vectores_temporales(timeseries)
mostrar_resumen_vectores(vectores_dynamic, indices_muestra=[0, 1, 2])

Dataframe 0: Extraídos 1370 registros
  V1 valores únicos: 2 - rango: 0 a 1
  day rango: 1 a 31
  weekday rango: 0 a 6
  week rango: 1 a 52
  month rango: 1 a 12
  quarter rango: 1 a 4
Dataframe 1: Extraídos 1282 registros
  V1 valores únicos: 2 - rango: 0 a 1
  day rango: 1 a 31
  weekday rango: 0 a 6
  week rango: 1 a 52
  month rango: 1 a 12
  quarter rango: 1 a 4
Dataframe 2: Extraídos 1374 registros
  V1 valores únicos: 2 - rango: 0 a 1
  day rango: 1 a 31
  weekday rango: 0 a 6
  week rango: 1 a 52
  month rango: 1 a 12
  quarter rango: 1 a 4
Dataframe 3: Extraídos 1370 registros
  V1 valores únicos: 2 - rango: 0 a 1
  day rango: 1 a 31
  weekday rango: 0 a 6
  week rango: 1 a 52
  month rango: 1 a 12
  quarter rango: 1 a 4
Dataframe 4: Extraídos 1372 registros
  V1 valores únicos: 2 - rango: 0 a 1
  day rango: 1 a 31
  weekday rango: 0 a 6
  week rango: 1 a 52
  month rango: 1 a 12
  quarter rango: 1 a 4
Dataframe 5: Extraídos 1360 registros
  V1 valores únicos: 2 - rango: 0 a 1

In [68]:
def extraer_primeros_indices(lista_dataframes):
    """
    Extrae el primer índice de cada dataframe y lo devuelve en formato "YYYY-MM-DD 00:00:00".
    
    Args:
        lista_dataframes: Lista de dataframes con índices de fecha
    
    Returns:
        Lista de strings con los primeros índices en formato "YYYY-MM-DD 00:00:00"
    """
    import pandas as pd
    from datetime import datetime
    
    primeros_indices = []
    
    for i, df in enumerate(lista_dataframes):
        # Verificar que el dataframe tenga registros
        if len(df) == 0:
            print(f"Advertencia: Dataframe {i} está vacío. Se usará fecha por defecto.")
            primeros_indices.append("2000-01-01 00:00:00")
            continue
        
        # Obtener el primer índice
        primer_indice = df.index[0]
        
        # Convertir a datetime si no lo es
        if not isinstance(primer_indice, pd.Timestamp) and not isinstance(primer_indice, datetime):
            try:
                primer_indice = pd.to_datetime(primer_indice)
            except:
                print(f"Advertencia: No se pudo convertir el índice del Dataframe {i} a fecha. Se usará fecha por defecto.")
                primeros_indices.append("2000-01-01 00:00:00")
                continue
        
        # Formatear a "YYYY-MM-DD 00:00:00"
        indice_formateado = primer_indice.strftime("%Y-%m-%d 00:00:00")
        
        primeros_indices.append(indice_formateado)
    
    return primeros_indices

In [69]:
start = extraer_primeros_indices(timeseries)

In [70]:
def crear_diccionarios_test(start, vectores_target, vectores_cat, vectores_dynamic):
    """
    Crea una lista de diccionarios con la estructura requerida para entrenamiento,
    donde start son las fechas de inicio de cada serie (un valor por dataframe).
    
    Args:
        start: Lista de strings con las fechas de inicio (un valor por dataframe)
        vectores_target: Lista de vectores con los valores de 'cantidad' para cada serie
        vectores_cat: Lista de vectores con las características categóricas
        vectores_dynamic: Lista de tuplas (V1, day, weekday, week, month, quarter) con características dinámicas
    
    Returns:
        Lista de diccionarios con la estructura {start, target, cat, dynamic_feat}
    """
    
    def convert_nans_to_string(values_list):
        """Convierte valores NaN en la lista a 'NaN' como string"""
        return ['NaN' if (isinstance(x, float) and np.isnan(x)) else float(x) for x in values_list]
    
    diccionarios = []
    
    # Verificar que tengamos el mismo número de series en todas las listas
    num_series = len(start)
    if not (num_series == len(vectores_target) == len(vectores_cat) == len(vectores_dynamic)):
        print(f"Error: Las listas tienen diferentes longitudes - start: {len(start)}, target: {len(vectores_target)}, " 
              f"cat: {len(vectores_cat)}, dynamic: {len(vectores_dynamic)}")
        return []
    
    for i in range(num_series):
        # Verificar que haya datos para esta serie
        if not vectores_target[i]:
            print(f"Advertencia: Serie {i} no tiene valores target. Se omitirá.")
            continue
        
        # Obtener los datos para esta serie
        fecha_inicio = start[i]
        target_data = vectores_target[i]
        cat_data = vectores_cat[i]
        
        # Extraer todos los vectores dinámicos de la tupla
        if len(vectores_dynamic[i]) == 6:
            V1_vector, day_vector, weekday_vector, week_vector, month_vector, quarter_vector = vectores_dynamic[i]
        else:
            print(f"Error: Serie {i} no tiene la estructura correcta de vectores dinámicos (esperados 6, encontrados {len(vectores_dynamic[i])})")
            continue
        
        # Verificar longitudes de vectores target y dynamic
        vectores_para_verificar = [V1_vector, day_vector, weekday_vector, week_vector, month_vector, quarter_vector]
        longitudes = [len(v) for v in vectores_para_verificar]
        
        if not all(len(target_data) == longitud for longitud in longitudes):
            print(f"Advertencia: Serie {i} tiene longitudes inconsistentes:")
            print(f"  target: {len(target_data)}")
            print(f"  V1: {len(V1_vector)}, day: {len(day_vector)}, weekday: {len(weekday_vector)}")
            print(f"  week: {len(week_vector)}, month: {len(month_vector)}, quarter: {len(quarter_vector)}")
        
        # Crear el diccionario
        diccionario = {
            "start": fecha_inicio,
            "target": convert_nans_to_string(target_data),
            "cat": cat_data,
            "dynamic_feat": [V1_vector, day_vector, weekday_vector, week_vector, month_vector, quarter_vector]
        }
        
        diccionarios.append(diccionario)
        
        # Información de diagnóstico
        print(f"Serie {i} procesada correctamente:")
        print(f"  Fecha inicio: {fecha_inicio}")
        print(f"  Longitud target: {len(target_data)}")
        print(f"  Características categóricas: {len(cat_data) if isinstance(cat_data, list) else 'valor único'}")
        print(f"  Vectores dinámicos: 6 vectores de longitud {len(V1_vector)}")
        print(f"  V1 - unos: {V1_vector.count(1)}, ceros: {V1_vector.count(0)}")
    
    print(f"\nTotal de diccionarios creados: {len(diccionarios)}")
    return diccionarios

In [71]:
test = crear_diccionarios_test(start, vectores_target, vectores_cat, vectores_dynamic)

Serie 0 procesada correctamente:
  Fecha inicio: 2021-08-14 00:00:00
  Longitud target: 1370
  Características categóricas: 5
  Vectores dinámicos: 6 vectores de longitud 1370
  V1 - unos: 54, ceros: 1316
Serie 1 procesada correctamente:
  Fecha inicio: 2021-08-20 00:00:00
  Longitud target: 1282
  Características categóricas: 5
  Vectores dinámicos: 6 vectores de longitud 1282
  V1 - unos: 54, ceros: 1228
Serie 2 procesada correctamente:
  Fecha inicio: 2021-08-13 00:00:00
  Longitud target: 1374
  Características categóricas: 5
  Vectores dinámicos: 6 vectores de longitud 1374
  V1 - unos: 54, ceros: 1320
Serie 3 procesada correctamente:
  Fecha inicio: 2021-08-13 00:00:00
  Longitud target: 1370
  Características categóricas: 5
  Vectores dinámicos: 6 vectores de longitud 1370
  V1 - unos: 54, ceros: 1316
Serie 4 procesada correctamente:
  Fecha inicio: 2021-08-14 00:00:00
  Longitud target: 1372
  Características categóricas: 5
  Vectores dinámicos: 6 vectores de longitud 1372
  V1

In [72]:
def crear_diccionarios_entrenamiento(start, vectores_target, vectores_cat, vectores_dynamic, puntos_a_excluir=30):
    """
    Crea una lista de diccionarios excluyendo los últimos 'puntos_a_excluir' valores de 
    target y dynamic_feat para cada serie.
    
    Args:
        start: Lista de strings con las fechas de inicio (un valor por dataframe)
        vectores_target: Lista de vectores con los valores de 'cantidad' para cada serie
        vectores_cat: Lista de vectores con las características categóricas
        vectores_dynamic: Lista de tuplas (V1, day, weekday, week, month, quarter) con características dinámicas
        puntos_a_excluir: Número de puntos a excluir del final de las series (default=6)
    
    Returns:
        Lista de diccionarios con la estructura {start, target, cat, dynamic_feat}
    """
    
    def convert_nans_to_string(values_list):
        """Convierte valores NaN en la lista a 'NaN' como string"""
        return ['NaN' if (isinstance(x, float) and np.isnan(x)) else float(x) for x in values_list]
    
    diccionarios = []
    
    # Verificar que tengamos el mismo número de series en todas las listas
    num_series = len(start)
    if not (num_series == len(vectores_target) == len(vectores_cat) == len(vectores_dynamic)):
        print(f"Error: Las listas tienen diferentes longitudes - start: {len(start)}, target: {len(vectores_target)}, " 
              f"cat: {len(vectores_cat)}, dynamic: {len(vectores_dynamic)}")
        return []
    
    print(f"Procesando {num_series} series para entrenamiento (excluyendo últimos {puntos_a_excluir} puntos)")
    
    for i in range(num_series):
        # Verificar que haya datos para esta serie
        if not vectores_target[i]:
            print(f"Advertencia: Serie {i} no tiene valores target. Se omitirá.")
            continue
        
        # Obtener los datos para esta serie
        fecha_inicio = start[i]
        target_data = vectores_target[i]
        cat_data = vectores_cat[i]
        
        # Extraer todos los vectores dinámicos de la tupla
        if len(vectores_dynamic[i]) == 6:
            V1_vector, day_vector, weekday_vector, week_vector, month_vector, quarter_vector = vectores_dynamic[i]
        else:
            print(f"Error: Serie {i} no tiene la estructura correcta de vectores dinámicos (esperados 6, encontrados {len(vectores_dynamic[i])})")
            continue
        
        # Verificar que hay suficientes puntos para excluir
        if len(target_data) <= puntos_a_excluir:
            print(f"Advertencia: Serie {i} tiene menos puntos ({len(target_data)}) que los requeridos a excluir ({puntos_a_excluir}). Se omitirá.")
            continue
        
        # Excluir los últimos 'puntos_a_excluir' valores de todos los vectores
        target_data_recortado = target_data[:-puntos_a_excluir]
        V1_vector_recortado = V1_vector[:-puntos_a_excluir]
        day_vector_recortado = day_vector[:-puntos_a_excluir]
        weekday_vector_recortado = weekday_vector[:-puntos_a_excluir]
        week_vector_recortado = week_vector[:-puntos_a_excluir]
        month_vector_recortado = month_vector[:-puntos_a_excluir]
        quarter_vector_recortado = quarter_vector[:-puntos_a_excluir]
        
        # Verificar longitudes después del recorte
        vectores_recortados = [
            V1_vector_recortado, day_vector_recortado, weekday_vector_recortado,
            week_vector_recortado, month_vector_recortado, quarter_vector_recortado
        ]
        nombres_vectores = ['V1', 'day', 'weekday', 'week', 'month', 'quarter']
        longitudes = [len(v) for v in vectores_recortados]
        
        # Verificar consistencia de longitudes
        if not all(len(target_data_recortado) == longitud for longitud in longitudes):
            print(f"Advertencia: Serie {i} tiene longitudes inconsistentes después del recorte:")
            print(f"  target: {len(target_data_recortado)}")
            for nombre, longitud in zip(nombres_vectores, longitudes):
                print(f"  {nombre}: {longitud}")
            
            # Ajustar a la longitud mínima
            min_len = min(len(target_data_recortado), *longitudes)
            print(f"  Ajustando todos los vectores a longitud: {min_len}")
            
            target_data_recortado = target_data_recortado[:min_len]
            vectores_recortados = [v[:min_len] for v in vectores_recortados]
            V1_vector_recortado, day_vector_recortado, weekday_vector_recortado, week_vector_recortado, month_vector_recortado, quarter_vector_recortado = vectores_recortados
        
        # Crear el diccionario
        diccionario = {
            "start": fecha_inicio,
            "target": convert_nans_to_string(target_data_recortado),
            "cat": cat_data,
            "dynamic_feat": [V1_vector_recortado, day_vector_recortado, weekday_vector_recortado, 
                           week_vector_recortado, month_vector_recortado, quarter_vector_recortado]
        }
        
        diccionarios.append(diccionario)
        
        # Información de diagnóstico
        print(f"Serie {i} procesada para entrenamiento:")
        print(f"  Longitud original: {len(target_data)} -> Longitud final: {len(target_data_recortado)}")
        print(f"  Puntos excluidos: {puntos_a_excluir}")
        print(f"  V1 en datos entrenamiento - unos: {V1_vector_recortado.count(1)}, ceros: {V1_vector_recortado.count(0)}")
    
    print(f"\nTotal de diccionarios de entrenamiento creados: {len(diccionarios)}")
    return diccionarios

In [73]:
train = crear_diccionarios_entrenamiento(start, vectores_target, vectores_cat, vectores_dynamic, puntos_a_excluir=30)

Procesando 27 series para entrenamiento (excluyendo últimos 30 puntos)
Serie 0 procesada para entrenamiento:
  Longitud original: 1370 -> Longitud final: 1340
  Puntos excluidos: 30
  V1 en datos entrenamiento - unos: 54, ceros: 1286
Serie 1 procesada para entrenamiento:
  Longitud original: 1282 -> Longitud final: 1252
  Puntos excluidos: 30
  V1 en datos entrenamiento - unos: 54, ceros: 1198
Serie 2 procesada para entrenamiento:
  Longitud original: 1374 -> Longitud final: 1344
  Puntos excluidos: 30
  V1 en datos entrenamiento - unos: 54, ceros: 1290
Serie 3 procesada para entrenamiento:
  Longitud original: 1370 -> Longitud final: 1340
  Puntos excluidos: 30
  V1 en datos entrenamiento - unos: 54, ceros: 1286
Serie 4 procesada para entrenamiento:
  Longitud original: 1372 -> Longitud final: 1342
  Puntos excluidos: 30
  V1 en datos entrenamiento - unos: 54, ceros: 1288
Serie 5 procesada para entrenamiento:
  Longitud original: 1360 -> Longitud final: 1330
  Puntos excluidos: 30
  V

In [74]:
def write_dicts_to_file(path, data):
    with open(path, "wb") as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode("utf-8"))

In [ ]:
%%time
write_dicts_to_file("data_json/diario/train.json", train)
write_dicts_to_file("data/json/diario/test.json", test)

CPU times: total: 46.9 ms
Wall time: 43.7 ms


In [76]:
boto_session = boto3.Session(profile_name='lilipink', region_name='us-east-1')
sagemaker_session = sagemaker.Session(boto_session=boto_session)
s3_client = boto_session.client('s3')
sm_client= boto_session.client('sagemaker')
s3 = boto_session.resource("s3")

[05/22/25 12:12:03] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=89412;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=733976;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\botocore\credentials.py#1352\1352]8;;\

In [77]:
def create_bucket(bucket_name, region=None):
    try:
        if region is None:
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            location = {'LocationConstraint': region}
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration=location
            )
        print(f"Bucket S3 '{bucket_name}' creado exitosamente en {region if region else 'la región por defecto'}")
        return True
    except ClientError as e:
        logging.error(e)
        print(f"Error al crear el bucket S3: {e}")
        return False

In [78]:
bucket_name = "forecasting-diario-27-v1"
create_bucket(bucket_name)

Bucket S3 'forecasting-diario-27-v1' creado exitosamente en la región por defecto


True

In [79]:
s3_bucket = bucket_name  # replace with an existing bucket if needed
s3_bucket_prefix = (
        "lilipink"  
    )
default_bucket_prefix = sagemaker_session.default_bucket_prefix
if default_bucket_prefix:
    s3_prefix = f"{default_bucket_prefix}/{s3_bucket_prefix}"
else:
    s3_prefix = s3_bucket_prefix

role = "arn:aws:iam::844598627082:role/service-role/AmazonSageMaker-ExecutionRole-20250513T105052"  # IAM role to use by SageMaker
region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output".format(s3_bucket, s3_prefix)

In [80]:
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", region)

[05/22/25 12:12:05] INFO     Same images used for training and inference. Defaulting to image     ]8;id=635868;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=918545;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\image_uris.py#393\393]8;;\
                             scope: inference.                                                                     

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=839339;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=413463;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\image_uris.py#530\530]8;;\

In [81]:
def copy_to_s3(local_file, s3_path, override=False):
    assert s3_path.startswith("s3://")
    split = s3_path.split("/")
    bucket = split[2]
    path = "/".join(split[3:])
    buk = s3.Bucket(bucket)

    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print(
                "File s3://{}/{} already exists.\nSet override to upload anyway.\n".format(
                    s3_bucket, s3_path
                )
            )
            return
        else:
            print("Overwriting existing file")
    with open(local_file, "rb") as data:
        print("Uploading file to {}".format(s3_path))
        buk.put_object(Key=path, Body=data)

In [ ]:
local_file = 'data_json/diario/'
copy_to_s3(local_file + 'train.json', s3_data_path + "/train/train.json",override=True)
copy_to_s3(local_file + 'test.json', s3_data_path + "/test/test.json",override=True)

Overwriting existing file
Uploading file to s3://forecasting-diario-27-v1/lilipink/data/train/train.json
Overwriting existing file
Uploading file to s3://forecasting-diario-27-v1/lilipink/data/test/test.json


In [83]:
#######OPTIMIZACION DE HYPERPARAMETROS###########

In [84]:
estimator = sagemaker.estimator.Estimator(
    image_uri=image_name,
    sagemaker_session=sagemaker_session,
    role=role,
    instance_count=1,
    instance_type="ml.c4.2xlarge",
   # use_spot_instances=True,
   # max_run=1800,  # max training time in seconds
   # max_wait=1800,  # seconds to wait for spot instance
    base_job_name="lilipink-forecasting",
    output_path=s3_output_path,
)

[05/22/25 12:12:07] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=633020;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=860717;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\botocore\credentials.py#1352\1352]8;;\

In [894]:
freq= "D"
context_length = 90
prediction_length = 30
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "likelihood": "student-T",
    #"learning_rate": "0.0001",
}
estimator.set_hyperparameters(**hyperparameters)

In [895]:
%%time
data_channels = {"train": "{}/train/".format(s3_data_path), "test": "{}/test/".format(s3_data_path)}

estimator.fit(inputs=data_channels, wait=True)

[05/22/25 10:32:57] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=117384;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=414108;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=892412;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=541686;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#1042\1042]8;;\
                             lilipink-forecasting-2025-05-22-15-32-57-063                                          

2025-05-22 15:32:58 Starting - Starting the training job...
2025-05-22 15:33:20 Starting - Preparing the instances for training...
2025-05-22 15:33:59 Downloading - Downloading the training image.........
2025-05-22 15:35:35 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
Running custom environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
[05/22/2025 15:35:50 INFO 140473021069120] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'_kvstore': 'auto', '_num_gpus': 'auto', '_num_kv_servers': 'auto', '_tuning_objective_metric': '', 'cardinality': 'auto', 'dropout_rate': '0.10', 'early_stopping_patience': '', 'embedding_dimension': '10', 'learning_rate': '0.001', 'likeli

In [85]:
estimator = sagemaker.estimator.Estimator.attach('lilipink-forecasting-2025-05-22-15-32-57-063',sagemaker_session=sagemaker_session)


2025-05-22 15:55:33 Starting - Preparing the instances for training
2025-05-22 15:55:33 Downloading - Downloading the training image
2025-05-22 15:55:33 Training - Training image download completed. Training in progress.
2025-05-22 15:55:33 Uploading - Uploading generated training model
2025-05-22 15:55:33 Completed - Training job completed


In [86]:
from sagemaker.serializers import IdentitySerializer

In [87]:
class DeepARPredictor(sagemaker.predictor.Predictor):
    def __init__(self, *args, **kwargs):
        super().__init__(
            *args,
            # serializer=JSONSerializer(),
            serializer=IdentitySerializer(content_type="application/json"),
            **kwargs,
        )

    def predict(
        self,
        ts,
        cat=None,
        dynamic_feat=None,
        num_samples=100,
        return_samples=False,
        quantiles=["0.1", "0.5", "0.9"],
    ):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.

        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])

        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + ts.index.freq
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)

    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(
            ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None
        )

        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles,
        }

        http_request_data = {"instances": [instance], "configuration": configuration}

        return json.dumps(http_request_data).encode("utf-8")

    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode("utf-8"))["predictions"][0]
        prediction_length = len(next(iter(predictions["quantiles"].values())))
        prediction_index = pd.date_range(
            start=prediction_time, freq=freq, periods=prediction_length
        )
        if return_samples:
            dict_of_samples = {"sample_" + str(i): s for i, s in enumerate(predictions["samples"])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(
            data={**predictions["quantiles"], **dict_of_samples}, index=prediction_index
        )

    def set_frequency(self, freq):
        self.freq = freq


def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]


def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat
    return obj

In [88]:
predictor = estimator.deploy(
    initial_instance_count=1, instance_type="ml.m5.large", predictor_cls=DeepARPredictor
)

[05/22/25 12:13:25] INFO     Creating model with name: lilipink-forecasting-2025-05-22-17-13-24-972 ]8;id=61649;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=148414;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=894669;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=809090;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#6019\6019]8;;\
                             lilipink-forecasting-2025-05-22-17-13-24-972                                          

[05/22/25 12:13:26] INFO     Creating endpoint with name                                            ]8;id=49765;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=849915;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4841\4841]8;;\
                             lilipink-forecasting-2025-05-22-17-13-24-972                                          

-------------!

In [89]:
def convertir_a_series(timeseries):
    series_list = []
    
    for ts in timeseries:
        # Verificar que la columna 'cantidad' existe
        if 'cantidad' in ts.columns:
            # Extraer la columna 'cantidad' como una serie
            serie = ts['cantidad']

            # Asegurarse de que el índice esté ordenado
            serie = serie.sort_index()

            # Intentar inferir la frecuencia del índice
            try:
                freq = pd.infer_freq(serie.index)
                if freq is not None:
                    serie.index.freq = freq
            except Exception as e:
                print(f"No se pudo inferir frecuencia para una serie: {e}")

            series_list.append(serie)
        else:
            print(f"Advertencia: Un dataframe no contiene la columna 'cantidad'")
    
    return series_list

In [90]:
timeseries_list = convertir_a_series(timeseries)

In [91]:
def crear_lista_features_dinamicas(lista_dataframes):
    """
    Crea una lista de listas con los vectores temporales para cada dataframe.
    
    Args:
        lista_dataframes: Lista de dataframes con columnas temporales
    
    Returns:
        Lista de listas donde cada elemento es [V1, day, weekday, week, month, quarter] para un dataframe
    """
    lista_features = []
    
    # Columnas temporales esperadas
    columnas_esperadas = ['V1', 'day', 'weekday', 'week', 'month', 'quarter']
    
    for i, df in enumerate(lista_dataframes):
        # Verificar que el dataframe tenga registros
        if len(df) == 0:
            print(f"Advertencia: Dataframe {i} está vacío. Se añadirá una lista vacía.")
            lista_features.append([[] for _ in columnas_esperadas])
            continue
        
        # Verificar que existan las columnas necesarias
        columnas_faltantes = []
        for col in columnas_esperadas:
            if col not in df.columns:
                columnas_faltantes.append(col)
        
        if columnas_faltantes:
            print(f"Advertencia: Dataframe {i} no tiene las columnas: {columnas_faltantes}")
            
            # Si faltan columnas, intentar generarlas a partir del índice si es posible
            df_temp = df.copy()
            
            # Generar columnas de fechas si el índice es de tipo datetime
            if isinstance(df_temp.index, pd.DatetimeIndex):
                if 'day' not in df_temp.columns:
                    df_temp['day'] = df_temp.index.day
                if 'weekday' not in df_temp.columns:
                    df_temp['weekday'] = df_temp.index.dayofweek
                if 'week' not in df_temp.columns:
                    df_temp['week'] = df_temp.index.isocalendar().week
                if 'month' not in df_temp.columns:
                    df_temp['month'] = df_temp.index.month
                if 'quarter' not in df_temp.columns:
                    df_temp['quarter'] = df_temp.index.quarter
                if 'V1' not in df_temp.columns:
                    # Crear V1 basado en el período 2023-09-10 a 2023-11-02
                    fecha_inicio_v1 = pd.Timestamp("2023-09-10")
                    fecha_fin_v1 = pd.Timestamp("2023-11-02")
                    df_temp['V1'] = 0
                    mask_v1 = (df_temp.index >= fecha_inicio_v1) & (df_temp.index <= fecha_fin_v1)
                    df_temp.loc[mask_v1, 'V1'] = 1
                
                df = df_temp
                print(f"  Columnas generadas automáticamente para Dataframe {i}")
                
            else:
                # Intentar convertir el índice a datetime si no lo es
                try:
                    df_temp.index = pd.to_datetime(df_temp.index)
                    
                    # Generar columnas después de convertir índice
                    if 'day' not in df_temp.columns:
                        df_temp['day'] = df_temp.index.day
                    if 'weekday' not in df_temp.columns:
                        df_temp['weekday'] = df_temp.index.dayofweek
                    if 'week' not in df_temp.columns:
                        df_temp['week'] = df_temp.index.isocalendar().week
                    if 'month' not in df_temp.columns:
                        df_temp['month'] = df_temp.index.month
                    if 'quarter' not in df_temp.columns:
                        df_temp['quarter'] = df_temp.index.quarter
                    if 'V1' not in df_temp.columns:
                        fecha_inicio_v1 = pd.Timestamp("2023-09-10")
                        fecha_fin_v1 = pd.Timestamp("2023-11-02")
                        df_temp['V1'] = 0
                        mask_v1 = (df_temp.index >= fecha_inicio_v1) & (df_temp.index <= fecha_fin_v1)
                        df_temp.loc[mask_v1, 'V1'] = 1
                    
                    df = df_temp
                    print(f"  Índice convertido a datetime y columnas generadas para Dataframe {i}")
                    
                except:
                    print(f"Error: No se pueden generar las columnas {columnas_faltantes} para el Dataframe {i}. Se añadirá una lista vacía.")
                    lista_features.append([[] for _ in columnas_esperadas])
                    continue
        
        # Crear los vectores de características dinámicas
        V1_vector = df['V1'].tolist()
        day_vector = df['day'].tolist()
        weekday_vector = df['weekday'].tolist()
        week_vector = df['week'].tolist()
        month_vector = df['month'].tolist()
        quarter_vector = df['quarter'].tolist()
        
        # Añadir los vectores a la lista en el orden correcto
        features_dataframe = [V1_vector, day_vector, weekday_vector, week_vector, month_vector, quarter_vector]
        lista_features.append(features_dataframe)
        
        # Información de diagnóstico para los primeros DataFrames
        if i < 5:
            print(f"Dataframe {i} procesado:")
            print(f"  Longitud de vectores: {len(V1_vector)}")
            print(f"  V1 - unos: {V1_vector.count(1)}, ceros: {V1_vector.count(0)}")
            print(f"  day rango: {min(day_vector)} - {max(day_vector)}")
            print(f"  weekday rango: {min(weekday_vector)} - {max(weekday_vector)}")
            print(f"  week rango: {min(week_vector)} - {max(week_vector)}")
            print(f"  month rango: {min(month_vector)} - {max(month_vector)}")
            print(f"  quarter rango: {min(quarter_vector)} - {max(quarter_vector)}")
    
    print(f"\nTotal de features dinámicas creadas: {len(lista_features)}")
    return lista_features

In [92]:
dynamic_list = crear_lista_features_dinamicas(timeseries)

Dataframe 0 procesado:
  Longitud de vectores: 1370
  V1 - unos: 54, ceros: 1316
  day rango: 1 - 31
  weekday rango: 0 - 6
  week rango: 1 - 52
  month rango: 1 - 12
  quarter rango: 1 - 4
Dataframe 1 procesado:
  Longitud de vectores: 1282
  V1 - unos: 54, ceros: 1228
  day rango: 1 - 31
  weekday rango: 0 - 6
  week rango: 1 - 52
  month rango: 1 - 12
  quarter rango: 1 - 4
Dataframe 2 procesado:
  Longitud de vectores: 1374
  V1 - unos: 54, ceros: 1320
  day rango: 1 - 31
  weekday rango: 0 - 6
  week rango: 1 - 52
  month rango: 1 - 12
  quarter rango: 1 - 4
Dataframe 3 procesado:
  Longitud de vectores: 1370
  V1 - unos: 54, ceros: 1316
  day rango: 1 - 31
  weekday rango: 0 - 6
  week rango: 1 - 52
  month rango: 1 - 12
  quarter rango: 1 - 4
Dataframe 4 procesado:
  Longitud de vectores: 1372
  V1 - unos: 54, ceros: 1318
  day rango: 1 - 31
  weekday rango: 0 - 6
  week rango: 1 - 52
  month rango: 1 - 12
  quarter rango: 1 - 4

Total de features dinámicas creadas: 27


In [93]:
horizon_pred=30
i=10
predictor.predict(
    ts = timeseries_list[i][:-horizon_pred], 
    cat=vectores_cat[i],
    dynamic_feat=dynamic_list[i],
    quantiles=[0.1, 0.5, 0.9],
).tail(6)

,0.1,0.5,0.9
2025-05-11,0.628661,8.023063,12.768902
2025-05-12,0.410730,4.412768,8.170145
2025-05-13,-0.486587,3.543970,8.228227
2025-05-14,-0.737101,4.278812,9.811383
2025-05-15,-0.043989,4.375904,10.301924
2025-05-16,0.811715,5.791229,11.961033


In [914]:
timeseries_list[i].tail(6)

fecha
2025-05-11     NaN
2025-05-12     1.0
2025-05-13     NaN
2025-05-14     3.0
2025-05-15     6.0
2025-05-16    12.0
Freq: D, Name: cantidad, dtype: float64

In [94]:
def generar_predicciones_por_material(materiales, timeseries_list, vectores_cat, dynamic_list, predictor, horizon_pred=6):
    """
    Genera predicciones para cada material y las devuelve en un diccionario.
    
    Args:
        materiales: Lista con los nombres de materiales
        timeseries_list: Lista de series temporales
        vectores_cat: Lista de vectores categóricos
        dynamic_list: Lista de features dinámicas
        predictor: Modelo predictor entrenado
        horizon_pred: Horizonte de predicción (default: 6)
    
    Returns:
        dict: Diccionario con materiales como keys y dataframes de predicciones como values
    """
    predicciones_dict = {}
    
    for i in range(len(materiales)):
        material = materiales[i]
        
        # Generar predicción para el índice i
        prediccion = predictor.predict(
            ts = timeseries_list[i][:-horizon_pred], 
            cat=vectores_cat[i],
            dynamic_feat=dynamic_list[i],
            quantiles=[0.1, 0.5, 0.9],
        )
        
        # Guardar en el diccionario
        predicciones_dict[material] = prediccion
        
    return predicciones_dict

In [95]:
predicciones_por_material = generar_predicciones_por_material(
    materiales=materiales,
    timeseries_list=timeseries_list,
    vectores_cat=vectores_cat,
    dynamic_list=dynamic_list,
    predictor=predictor,
    horizon_pred=30
)

In [921]:
def exportar_predicciones_consolidado(predicciones_dict, nombre_archivo="diario_modificado_test_27.xlsx"):
    """
    Exporta todas las predicciones con Material y Fecha como primeras dos columnas.
    """
    
    dfs_list = []
    
    for material, prediccion in predicciones_dict.items():
        try:
            # Convertir a DataFrame
            if hasattr(prediccion, 'to_dataframe'):
                df_pred = prediccion.to_dataframe()
            elif hasattr(prediccion, 'to_pandas'):
                df_pred = prediccion.to_pandas()
            else:
                df_pred = prediccion
            
            # Resetear índice y renombrar a 'Fecha'
            df_pred = df_pred.reset_index()
            date_col = df_pred.columns[0]
            df_pred = df_pred.rename(columns={date_col: 'Fecha'})
            
            # Formatear fechas
            df_pred['Fecha'] = pd.to_datetime(df_pred['Fecha']).dt.strftime('%Y-%m-%d')
            
            # Agregar columna de material
            df_pred['Material'] = material
            
            dfs_list.append(df_pred)
            
        except Exception as e:
            print(f"Error procesando {material}: {e}")
    
    # Concatenar todos los DataFrames
    if dfs_list:
        df_final = pd.concat(dfs_list, ignore_index=True)
        
        # Reorganizar columnas: Material, Fecha, luego el resto en orden
        other_cols = [col for col in df_final.columns if col not in ['Material', 'Fecha']]
        cols = ['Material', 'Fecha'] + other_cols
        df_final = df_final[cols]
        
        # Exportar
        df_final.to_excel(nombre_archivo, index=False)
        print(f"Archivo guardado: {nombre_archivo}")
        print(f"Orden de columnas: {list(df_final.columns)}")
        print("Formato de fecha: YYYY-MM-DD")
    else:
        print("Error: No se pudieron procesar las predicciones")

# Ejecutar
exportar_predicciones_consolidado(predicciones_por_material, "diario_modificado_test_27.xlsx")

Archivo guardado: diario_modificado_test_27.xlsx
Orden de columnas: ['Material', 'Fecha', '0.1', '0.5', '0.9']
Formato de fecha: YYYY-MM-DD


In [96]:
predictor.delete_model()
predictor.delete_endpoint()

[05/22/25 12:29:35] INFO     Deleting model with name: lilipink-forecasting-2025-05-22-17-13-24-972 ]8;id=348816;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=988575;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#5356\5356]8;;\

                    INFO     Deleting endpoint configuration with name:                             ]8;id=802946;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=205363;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4995\4995]8;;\
                             lilipink-forecasting-2025-05-22-17-13-24-972                                          

                    INFO     Deleting endpoint with name:                                           ]8;id=47534;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=722365;file://c:\Users\Usuario\Desktop\Proyecto_lilipink\venv\Lib\site-packages\sagemaker\session.py#4985\4985]8;;\
                             lilipink-forecasting-2025-05-22-17-13-24-972                                          